# Imports

In [4]:
%matplotlib inline

from IPython import get_ipython
ipython = get_ipython()
ipython.magic("gui qt5") 

import napari
from PIL import Image

import numpy as np
import cv2

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.colors as colors
import matplotlib.path as mpltpath

import time

import pickle as pkl
#import h5py
from os.path import sep

from bisect import bisect_left

# import Cell
# import pixels_in_roi
# import diff_gauss
# import scatter_channels
# import hist_channels
# import m_dist

from skimage.measure import profile_line
from skimage.filters import gaussian


# Load images

## Raw images

### Specify planes

In [49]:
t0 = time.time()
folder = 'G:{0}Shared drives{0}as_share{0}HCR{0}HCR 3.11{0}S5{0}S5_raw_tiff_images'.format(sep)
plane_nos = range(1, 11)

# base_filename2 = 'S1_dapi_546_488_647_s2z'
base_filename = 'S5__z'
n = len(plane_nos)
print('Number of planes: {0}'.format(n))

# Create 4D array to store images
img = Image.open('{0}{1}{2}{3}_ChS1-T3_ORG.tif'.format(folder, sep, base_filename, str(plane_nos[0]).zfill(2)))

h = img.height
w = img.width

Number of planes: 10


### Load images

In [110]:
t0 = time.time()
folder = 'G:{0}Shared drives{0}as_share{0}HCR{0}HCR 3.11{0}S5{0}S5_raw_tiff_images'.format(sep)
plane_nos = range(1, 2)

# base_filename2 = 'S1_dapi_546_488_647_s2z'
base_filename = 'S5__z'
n = len(plane_nos)
print('Number of planes: {0}'.format(n))

# Create 4D array to store images
img = Image.open('{0}{1}{2}{3}_ChS1-T3_ORG.tif'.format(folder, sep, base_filename, str(plane_nos[0]).zfill(2)))

h = img.height
w = img.width
im_array_gad1 = np.zeros([n, h, w])
# im_array_vip = np.zeros([n, h, w])
# im_array_sst = np.zeros([n, h, w])
# im_array_ndnf = np.zeros([n, h, w])
#im_array_dapi = np.zeros([n, h, w])

print('Size of image in pixels: {0} X {1} X {2}'.format(n, h, w))

for i in range(n):
    print('Loading image {0}, {1} seconds'.format(i + 1, np.round(time.time() - t0)))
    p = plane_nos[i]
    img_gad1 = Image.open('{0}{1}{2}{3}_ChS1-T3_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
#     img_vip = Image.open('{0}{1}{2}{3}_ChS2-T3_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
#     img_sst = Image.open('{0}{1}{2}{3}_ChS2-T2_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
#     img_ndnf = Image.open('{0}{1}{2}{3}_ChS2-T1_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
    #img_dapi = Image.open('{0}{1}{2}{3}_ChS1-T1_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))

    try:
        im_array_gad1[i, :, :] = np.array(img_gad1)
#         im_array_vip[i, :, :] = np.array(img_vip)
#         im_array_sst[i, :, :] = np.array(img_sst)
#         im_array_ndnf[i, :, :] = np.array(img_ndnf)
        #im_array_dapi[i, :, :] = np.array(img_dapi)
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        im_array_gad1 = np.delete(im_array_gad1, i, axis = 0)
        im_array_vip = np.delete(im_array_vip, i, axis = 0)
        im_array_sst = np.delete(im_array_sst, i, axis = 0)
        im_array_ndnf = np.delete(im_array_ndnf, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue

del img_gad1
# del img_vip
# del img_sst
# del img_ndnf
#del img_dapi

Number of planes: 1
Size of image in pixels: 1 X 3788 X 14847
Loading image 1, 0.0 seconds


## Gaussian blurred images

In [3]:
# %% Load images
t0 = time.time()
folder = 'G:{0}Shared drives{0}as_share{0}HCR{0}HCR 3.11{0}S5{0}S5_blurred_tiff_images'.format(sep)
plane_nos = range(1, 11)

# base_filename2 = 'S1_dapi_546_488_647_s2z'
base_filename = 'S5_z'
n = len(plane_nos)
print('Number of planes: {0}'.format(n))

# Create 4D array to store images
img = Image.open('{0}{1}{2}{3}_Gad1_5px_gauss_blur.tif'.format(folder, sep, base_filename, str(plane_nos[0]).zfill(2)))

h = img.height
w = img.width
im_array_gad1_blur = np.zeros([n, h, w])
# im_array_vip = np.zeros([n, h, w])
# im_array_sst = np.zeros([n, h, w])
# im_array_ndnf = np.zeros([n, h, w])
# im_array_dapi = np.zeros([n, h, w])

print('Size of image in pixels: {0} X {1} X {2}'.format(n, h, w))

for i in range(n):
    print('Loading image {0}, {1} seconds'.format(i + 1, np.round(time.time() - t0)))
    p = plane_nos[i]
    img_gad1 = Image.open('{0}{1}{2}{3}_Gad1_5px_gauss_blur.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
#     img_vip = Image.open('{0}{1}{2}{3}_ChS2-T3_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
#     img_sst = Image.open('{0}{1}{2}{3}_ChS2-T2_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
#     img_ndnf = Image.open('{0}{1}{2}{3}_ChS2-T1_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))
#     img_dapi = Image.open('{0}{1}{2}{3}_ChS1-T1_ORG.tif'.format(folder, sep, base_filename, str(p).zfill(2)))

    try:
        im_array_gad1_blur[i, :, :] = np.array(img_gad1)
#         im_array_vip[i, :, :] = np.array(img_vip)
#         im_array_sst[i, :, :] = np.array(img_sst)
#         im_array_ndnf[i, :, :] = np.array(img_ndnf)
#         im_array_dapi[i, :, :] = np.array(img_dapi)
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        im_array_gad1 = np.delete(im_array_gad1, i, axis = 0)
        im_array_vip = np.delete(im_array_vip, i, axis = 0)
        im_array_sst = np.delete(im_array_sst, i, axis = 0)
        im_array_ndnf = np.delete(im_array_ndnf, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue

del img_gad1
# del img_vip
# del img_sst
# del img_ndnf
# del img_dapi

Number of planes: 10
Size of image in pixels: 10 X 3788 X 14847
Loading image 1, 0.0 seconds
Loading image 2, 1.0 seconds
Loading image 3, 2.0 seconds
Loading image 4, 4.0 seconds
Loading image 5, 5.0 seconds
Loading image 6, 6.0 seconds
Loading image 7, 7.0 seconds
Loading image 8, 8.0 seconds
Loading image 9, 9.0 seconds
Loading image 10, 11.0 seconds


## Background subtracted (rolling ball)

In [ ]:
# Background subtracted (using rolling ball background subtraction in Fiji)
print('Ndnf')
im_array_ndnf = np.zeros([n, h, w])
img_file = 'S2_Ndnf_bg_sub2'
img = Image.open('{0}\{1}.tif'.format(folder, img_file))
for i in range(n):
    print('Loading image {0}'.format(i + 1))
    p = plane_nos[i]
    try:
        img.seek(p - 1)
        im_array_ndnf[i, :, :] = np.array(img)
        
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        im_array_ndnf = np.delete(im_array_ndnf, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue

print('Vip')
im_array_vip = np.zeros([n, h, w])
img_file = 'S2_Vip_bg_sub2'
img = Image.open('{0}\{1}.tif'.format(folder, img_file))
for i in range(n):
    print('Loading image {0}'.format(i + 1))
    p = plane_nos[i]
    try:
        img.seek(p - 1)
        im_array_vip[i, :, :] = np.array(img)
        
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        im_array_vip = np.delete(im_array_vip, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue
        
        
print('Sst')
im_array_sst = np.zeros([n, h, w])
img_file = 'S2_Sst_bg_sub2'
img = Image.open('{0}\{1}.tif'.format(folder, img_file))
for i in range(n):
    print('Loading image {0}'.format(i + 1))
    p = plane_nos[i]
    try:
        img.seek(p - 1)
        im_array_sst[i, :, :] = np.array(img)
        
    except:
        print('Plane {0} could not be loaded'.format(p))
        print('Size of plane {0} in pixels: {0} X {1}'.format(img.height, img.width))
        im_array_sst = np.delete(im_array_sst, i, axis = 0)
        plane_nos.remove(p)
        i -= 1
        n = len(plane_nos)
        continue    

# Render images in napari GUI

## Raw images

In [111]:
# Full images
t0 = time.time()

# viewer = napari.Viewer()
viewer.add_image(im_array_gad1, name = 'Gad1', colormap = 'cyan', blending = 'additive')
# viewer.add_image(im_array_ndnf, name = 'Ndnf', colormap = 'red', blending = 'additive')
# viewer.add_image(im_array_vip, name = 'Vip', colormap = 'green', blending = 'additive')
# viewer.add_image(im_array_sst, name = 'Sst', colormap = 'blue', blending = 'additive')

t1 = time.time() - t0
print('{0} seconds'.format(np.round(t1)))

1.0 seconds


## Blurred images

In [9]:
viewer.add_image(im_array_gad1_blur, name = 'Gad1_blur', colormap = 'cyan', blending = 'additive')

<Image layer 'Gad1_blur' at 0x15913608>

## Cell masks

In [ ]:
# Cells with high mask variance
var_thresh = 600
indices = np.where(np.logical_or(var_masks[:, 0] > var_thresh, var_masks[:, 1] > var_thresh))[0] + 1
print(indices)

In [112]:
mask_layer2 = viewer.add_shapes(name = 'Cell masks individual')
mask_cell_nos = []
cell_mask_nos = {}
t0 = time.time()

#indices = list(Cell.cell_data.keys())
indices = list(Cell.cell_data.keys())[1:100]
for cell in indices:
    if np.mod(cell, 10) == 0:
        print('Cell {0}, {1} seconds'.format(cell, np.round(time.time() - t0)))
    planes = Cell.cell_data[cell]['z_planes']
    cell_mask_nos[cell] = []
    for plane in planes:
        mask = Cell.cell_data[cell]['masks'][plane]
        mask = np.concatenate((np.ones([mask.shape[0], 1])*plane, mask), axis = 1)
        mask_layer2.add(mask, shape_type = 'polygon', opacity = 0.4, 
                        face_color = 'white', edge_color = 'red', edge_width = 5)
        cell_mask_nos[cell] = np.append(cell_mask_nos[cell], len(mask_cell_nos) - 1)
        mask_cell_nos = np.append(mask_cell_nos, cell)

Cell 10, 0.0 seconds
Cell 20, 0.0 seconds
Cell 30, 1.0 seconds
Cell 40, 2.0 seconds
Cell 50, 2.0 seconds
Cell 60, 3.0 seconds
Cell 70, 3.0 seconds
Cell 80, 4.0 seconds
Cell 90, 5.0 seconds
Cell 100, 6.0 seconds


## Lipofuscin ROIs (annotated)

In [9]:
n_rois = len(l_rois)
print('{0} rois found'.format(n_rois))
viewer.add_shapes(l_rois, name = 'Lipofuscin ROIs',
                   shape_type = 'polygon', opacity = 1, face_color = 'white', 
                   edge_color = 'blue', edge_width = 3)

14 rois found


<Shapes layer 'Lipofuscin ROIs' at 0x4bf3eb48>

## Filtered images

### Filtering for intensity calculation

In [46]:
viewer.add_image(data = im_diff_ndnf, name = 'Diff ndnf', colormap = 'green', blending = 'additive')
viewer.add_image(data = im_diff_sst, name = 'Diff Sst', colormap = 'magenta', blending = 'additive')
viewer.add_image(data = im_diff_vip, name = 'Diff Vip', colormap = 'cyan', blending = 'additive')
# viewer.add_image(data = im_diff_gad1, name = 'Diff Gad1', colormap = 'cyan', blending = 'additive')

# viewer.add_image(data = im_diff_ndnf[0, :, :], name = 'Diff ndnf', colormap = 'magenta', blending = 'additive')
# viewer.add_image(data = im_diff_sst[0, :, :], name = 'Diff Sst', colormap = 'green', blending = 'additive')
# viewer.add_image(data = im_diff_vip[0, :, :], name = 'Diff Vip', colormap = 'yellow', blending = 'additive')
# #viewer.add_image(data = im_diff_gad1[0, :, :], name = 'Diff Gad1', colormap = 'cyan', blending = 'additive')

<Image layer 'Diff Vip' at 0x16e118c8>

### Filtering for lipofuscin detection (larger mexican hat filter)

In [85]:
viewer.add_image(data = im_diff_ndnf_lipo, name = 'Diff ndnf_lipo', colormap = 'magenta', blending = 'additive')
viewer.add_image(data = im_diff_sst_lipo, name = 'Diff Sst_lipo', colormap = 'green', blending = 'additive')
viewer.add_image(data = im_diff_vip_lipo, name = 'Diff Vip_lipo', colormap = 'yellow', blending = 'additive')

<Image layer 'Diff Vip_lipo' at 0x6736e1c8>

## Binarized images

### For intensity calculation

In [66]:
viewer.add_image(data = im_bin_ndnf[4, :, :], name = 'Binary ndnf', colormap = 'green', blending = 'additive')
viewer.add_image(data = im_bin_sst[4, :, :], name = 'Binary Sst', colormap = 'magenta', blending = 'additive')
viewer.add_image(data = im_bin_vip[4, :, :], name = 'Binary Vip', colormap = 'cyan', blending = 'additive')

<Image layer 'Binary Vip' at 0x1998bc88>

### For lipofuscin detection

In [86]:
viewer.add_image(data = im_bin_ndnf_lipo, name = 'Binary ndnf lipo', colormap = 'magenta', blending = 'additive')
viewer.add_image(data = im_bin_sst_lipo, name = 'Binary Sst lipo', colormap = 'green', blending = 'additive')
viewer.add_image(data = im_bin_vip_lipo, name = 'Binary Vip lipo', colormap = 'yellow', blending = 'additive')

<Image layer 'Binary Vip lipo' at 0x604debc8>

## Detected lipofuscin pixels

In [84]:
viewer.add_image(data = im_array_lipo, name = 'Detected lipofuscin', colormap = 'gray', blending = 'additive')

<Image layer 'Detected lipofuscin [1]' at 0x51297dc8>

## Cell masks after removing lipofuscin (check)

In [41]:
im_array_cells = np.zeros(im_array_ndnf.shape)


indices = list(Cell.cell_data.keys())
for cell in indices:
    if np.mod(cell, 10) == 0:
        print('Cell {0}'.format(cell))
    planes = Cell.cell_data[cell]['z_planes']
    for plane in planes:
        if plane + 1 in list(plane_nos):
            mask = cell_pixels_no_lipo[cell][plane]
            mask = np.concatenate((np.ones([1, mask.shape[1]])*plane, mask))
            mask = mask.astype(int)
            im_array_cells[mask[0, :], mask[1, :], mask[2, :]] = np.ones(mask.shape[1])
        

viewer.add_image(data = im_array_cells, name = 'Cell pixels', colormap = 'blue', blending = 'additive')

Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280
Cell 290
Cell 300
Cell 310
Cell 320
Cell 330
Cell 340
Cell 350


<Image layer 'Cell pixels' at 0x608a8208>

In [39]:
list(plane_nos)

[1, 2]

## Cells with significant expression

In [93]:
ndnf_mask_layer = viewer.add_shapes(name = 'Ndnf+ cells')

#indices = list(Cell.cell_data.keys())
indices = [120]
for cell in indices:
    
    if pv_ndnf3[cell] > 0.05:
        planes = Cell.cell_data[cell + 1]['z_planes']
        for plane in planes:
            mask = Cell.cell_data[cell + 1]['masks'][plane]
            mask = np.concatenate((np.ones([mask.shape[0], 1])*plane, mask), axis = 1)
            ndnf_mask_layer.add(mask, shape_type = 'polygon', opacity = 0.2, face_color = 'white', edge_color = 'magenta', edge_width = 3)

In [101]:
indices = [82]
vip_mask_layer = viewer.add_shapes(name = 'Vip+ cells')

#indices = list(Cell.cell_data.keys())

for cell in indices:
    
    if pv_vip3[cell] > 0.05:
        planes = Cell.cell_data[cell + 1]['z_planes']
        for plane in planes:
            mask = Cell.cell_data[cell + 1]['masks'][plane]
            mask = np.concatenate((np.ones([mask.shape[0], 1])*plane, mask), axis = 1)
            vip_mask_layer.add(mask, shape_type = 'polygon', opacity = 0.2, face_color = 'white', edge_color = 'yellow', edge_width = 3)

In [98]:
indices = [132]
sst_mask_layer = viewer.add_shapes(name = 'Sst+ cells')

#indices = list(Cell.cell_data.keys())

for cell in indices:
    
    if pv_sst3[cell] < 0.05:
        planes = Cell.cell_data[cell + 1]['z_planes']
        for plane in planes:
            mask = Cell.cell_data[cell + 1]['masks'][plane]
            mask = np.concatenate((np.ones([mask.shape[0], 1])*plane, mask), axis = 1)
            sst_mask_layer.add(mask, shape_type = 'polygon', opacity = 0.2, face_color = 'white', edge_color = 'green', edge_width = 3)

# Cell segmentation

## Load existing masks

In [3]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11'
save_file= 'S5_data.pkl'
try:
    with open('{0}\{1}'.format(save_loc, save_file), 'rb') as f:
        Cell.cell_data = pkl.load(f)
        indices = list(Cell.cell_data.keys())
        if not np.max(indices) == len(indices):
            print('Re-numbering cells to be consecutive')
            Cell.cell_data_temp = {}
            for i in range(len(indices)):
                Cell.cell_data_temp[i + 1] = Cell.cell_data[indices[i]]
                Cell.cell_data_temp[i + 1]['cell_id'] = i + 1
            Cell.cell_data = Cell.cell_data_temp 
            with open('{0}\{1}'.format(save_loc, save_file), 'wb') as f:
                pkl.dump(Cell.cell_data, f)
            Cell.n_cells = i + 1
        else:
            Cell.n_cells = len(indices)
    print('{0} cells found'.format(Cell.n_cells))
except:
    print('No data found')

357 cells found


## If loading a subset of planes, sub-select cells

In [5]:
cells_sub = []
for cell in range(Cell.n_cells):
    flag = 0
    cell_no = cell + 1
    cp = Cell.cell_data[cell_no]['z_planes']
    for p in cp:
        if p + 1 not in list(plane_nos):
            flag = 1
    if flag == 0:
        cells_sub = np.append(cells_sub, cell)

cells_sub = np.array(cells_sub)
cells_sub = cells_sub.astype(int)
print(cells_sub)

[  0   1   2   3   4   5   6   7   8   9  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  28  30  31  32  33  34  35  37  38  39
  40  41  42  43  44  48  49  50  51  52  53  54  55  56  57  58  59  60
  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78
  80  81  82  83  84  85  86  87  88  89  90  91  92  93  95  96  97  99
 102 104 105 106 110 111 120 125 132 140 141 143 145 146 165]


## Get pixels in cell masks

In [6]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\S5'
#save_file = 'S2_cell_pixels.pkl'
save_file = 'S5_mask_vertices.pkl'

t0 = time.time()

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        mask_vertices = pkl.load(f)
        cells = mask_vertices.keys()
#         cells = cells_sub
        x = 0
        all_cell_pixels = np.zeros([1, 3])
        for cell in cells:
            cell_no = cell
#             cell_no = cell + 1
            data = Cell.cell_data[cell_no]
            planes = data['z_planes']
            for plane in planes:
                xvals = np.reshape(mask_vertices[cell_no][plane][0], [-1, 1])
                yvals = np.reshape(mask_vertices[cell_no][plane][1], [-1, 1])
                zvals = np.ones([len(xvals), 1])*plane
                coords = np.concatenate((zvals, xvals, yvals), axis = 1)
                all_cell_pixels = np.concatenate((all_cell_pixels, coords), axis = 0)
        
        all_cell_pixels = all_cell_pixels[1:, :].astype(int)
            
        print('Data loaded')
        
except IOError:
    print('No saved data found, calculating mask pixels')
#     xv = range(w)
#     yv = range(h)
#     coord_array = np.array(np.meshgrid(xv, yv))

#     points = np.zeros([h*w, 2])
#     p = 0
#     for i in range(h):
#         for j in range(w):
#             points[p, 1] = coord_array[0, i, j]
#             points[p, 0] = coord_array[1, i, j]
#             p += 1

    no_cells = len(Cell.cell_data.keys())
    mask_vertices = {}
    for cell in range(no_cells):
        if np.mod(cell, 2) == 0:
            print('Cell {0}: {1} seconds'.format(cell, np.round(time.time() - t0)))
        cell_no = cell + 1
        mask_vertices[cell_no] = {}
        cell_dict = Cell.cell_data[cell_no]
        masks = cell_dict['masks']
        z_planes = cell_dict['z_planes']
        for plane in z_planes:

            vertices = masks[plane]
            path = mpltpath.Path(vertices)
            mask = path.contains_points(points)
            mask = np.reshape(mask, [h, w])
            mask_vertices[cell_no][plane] = np.where(mask)
    
    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(mask_vertices, f)

NameError: name 'Cell' is not defined

In [35]:
with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
    pkl.dump(mask_vertices, f)

## Semi-manual segmentation using Gad1 channel

### Data structure to store cell masks

In [5]:
class Cell:
    n_cells = 1
    cell_data = {}
    
    def __init__(self, cell_no):
        
        self.cell_no = cell_no
        self.z_planes = []
        self.no_planes = 0
        self.masks = {}
        self.c_vals = np.zeros(1)
        print('Cell #{0} created'.format(Cell.n_cells))
        
    def add_mask(self, z_plane, mask):
        self.z_planes.append(z_plane)
        self.no_planes += 1
        self.masks[z_plane] = mask
            
    def save_cell(self):
        Cell.cell_data[self.cell_no] = {'cell_id': self.cell_no,
                                 'z_planes': self.z_planes,
                                 'masks': self.masks,
                                 'no_planes': self.no_planes,
                                 'c_vals':self.c_vals}

### Napari keybindings

In [10]:
mask_layer = viewer.add_shapes(data = None, shape_type = 'polygon', opacity = 0.2, face_color = 'white', edge_color = 'red', edge_width = 3)
point_layer = viewer.add_points(data = None)
save_loc = 'G:{0}Shared drives{0}as_share{0}HCR{0}HCR 3.11'.format(sep)
save_file = 'S5_data.pkl'

@viewer.bind_key('n', overwrite = True)
def new_cell(viewer):
    global cell1 
    cell1 = Cell(Cell.n_cells)
    print(Cell.n_cells)
    Cell.n_cells += 1
    
    
@viewer.bind_key('d', overwrite = True)
def get_mask(viewer):
    center = point_layer.data[-1, 1:] # Last drawn point
    z_plane = point_layer.data[-1, 0].astype(int)
    image = im_array_ndnf_blur[z_plane, :, :]
    cell_radius_px = 70

    boundary = disc_roi(image, center, z_plane, cell_radius_px)

    mask_layer.add(boundary, shape_type= 'polygon')
    
    
@viewer.bind_key('m', overwrite = True)
def add_mask(viewer):
    
    z_plane = mask_layer.data[-1][0, 0]
    mask = mask_layer.data[-1][:, 1:]
    cell1.add_mask(z_plane, mask)
    
@viewer.bind_key('y', overwrite = True)
def del_cell(viewer):
    
    mask_no = mask_layer2.selected_data[0]
    cell = mask_cell_nos[mask_no]
    z_plane = mask_layer2.coordinates[0]
    
    Cell.cell_data[cell]['masks'][z_plane] = mask_layer2.data[mask_no][:, 1:]
    with open('{0}\{1}'.format(save_loc, save_file), 'wb') as f:
                pkl.dump(Cell.cell_data, f)
    
    mask = Cell.cell_data[cell]['masks'][z_plane]
    n_vertices = mask.shape[0]
    mask = np.concatenate([np.ones([n_vertices, 1])*z_plane, mask], axis = 1)
    mask_layer2.add(mask, shape_type = 'polygon', opacity = 0.4, 
                    face_color = 'white', edge_color = 'yellow', edge_width = 5)
    
    
@viewer.bind_key('k', overwrite = True)
def save_cell(viewer):
    cell1.save_cell()
    
@viewer.bind_key('Control-s', overwrite = True)
def save_all(viewer):
    with open('{0}\{1}'.format(save_loc, save_file), 'wb') as f:
                pkl.dump(Cell.cell_data, f)

In [12]:
del_cell(viewer)

IndexError: list index out of range

### One click segmentation

#### View saved masks

In [161]:
Cell.n_cells

357

In [72]:
with open('{0}\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(Cell.cell_data, f)

In [13]:
# Check number of masks per cell, and variance of mask centers
indices = list(Cell.cell_data.keys())
n_planes = np.zeros(len(indices)).astype(int)
var_masks = np.zeros([len(indices), 2])

if not np.max(indices) == len(indices):
    print('{0} cells not recorded'.format(np.max(indices) - len(indices)))
    print('Re-numbering cells to be consecutive')
    Cell.cell_data_temp = {}
    for i in range(len(indices)):
        Cell.cell_data_temp[i + 1] = Cell.cell_data[indices[i]]
        Cell.cell_data_temp[i + 1]['cell_id'] = i + 1
    Cell.cell_data = Cell.cell_data_temp 
    with open('{0}\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(Cell.cell_data, f)
        
Cell.n_cells = len(indices)
        
for i in range(len(indices)):
    z_planes = Cell.cell_data[i + 1]['z_planes']
    n_planes[i] = len(z_planes)
    masks = Cell.cell_data[i + 1]['masks']
    centers = np.zeros([n_planes[i], 2])
    for j in range(n_planes[i]):
        centers[j, 0] = np.mean(masks[z_planes[j]][:, 0])
        centers[j, 1] = np.mean(masks[z_planes[j]][:, 1])
    var_masks[i, 0] = np.var(centers[:, 0])
    var_masks[i, 1] = np.var(centers[:, 1])
    
    

In [14]:
fig, ax = plt.subplots(nrows = 2, ncols = 1, sharex = True, figsize = [15, 5])
ax[0].plot(n_planes)
ax[0].set_xlabel('Cell number')
ax[0].set_ylabel('Number of masks')
ax[1].plot(var_masks[:, 0], label = 'y_values')
ax[1].plot(var_masks[:, 1], label = 'x_values')
ax[1].set_ylabel('Variance of centers')
ax[1].plot(range(len(indices)), np.ones(len(indices))*8, '--', color = 'k')
ax[1].legend()

#### Functions

In [9]:
def disc_roi(image, center, z_plane, cell_radius_px, n_theta = 30, thresh = 0.5):
    
    full_radius = round(1.3*cell_radius_px)
    thetas = np.linspace(0, 2*np.pi, n_theta)
    [x, y] = center
    
    
    line_profiles = np.zeros([full_radius, n_theta])
    
    r_threshold_cross = np.zeros(n_theta)
    
    for t in range(n_theta):
        theta = thetas[t]
        f = profile_line(image, (x, y), (x + full_radius*np.cos(theta), y + full_radius*np.sin(theta)))
        r_threshold_cross[t] = find_first_cross(f, thresh)
        line_profiles[:, t] = -1*np.abs(np.array(range(full_radius)) - r_threshold_cross[t]) + full_radius;
        
    #path = find_path(line_profiles)
   
    #boundary = np.zeros([2, n_theta])
    #boundary[0, :] = x + np.multiply(path, np.cos(thetas))
    #boundary[1, :] = y + np.multiply(path, np.sin(thetas))
    #boundary = boundary.astype(int)
    
    angular_dist = np.argmax(line_profiles, axis = 0)
    boundary = np.zeros([n_theta, 3])
    boundary[:, 0] = np.ones(n_theta)*z_plane
    boundary[:, 1] = x + np.multiply(angular_dist, np.cos(thetas))
    boundary[:, 2] = y + np.multiply(angular_dist, np.sin(thetas))
    
    
    
    return boundary

def find_first_cross(line_profile, thresh):
    
    # Assumes line profile goes from inside cell to outside
    max_val = np.max(line_profile)
    max_idx = np.argmax(line_profile)
    min_val_out = np.min(line_profile[max_idx:]) # Minimum intensity outside cell
    
    thresh_val = min_val_out + thresh*(max_val - min_val_out)
    r_cross = max_idx + np.where(line_profile[max_idx:] < thresh_val)[0][0]
    
    return r_cross

def find_path(line_profiles):
    
    # line_profiles should be of shape radius X n_theta
    radius = line_profiles.shape[0]
    n_theta = line_profiles.shape[1]
    
    pointer = np.zeros([radius, n_theta])
    value = np.zeros([radius, n_theta])
    value[:, 0] = line_profiles[:, 0]
    
    for i in range(1, n_theta):
        for j in range(1, radius - 1):
            
            M = np.max(value[j-1:j+1, i-1])
            ind = np.argmax(value[j-1:j+1, i-1])
            
            value[j, i] = M + line_profiles[j, i]
            pointer[j,i] = j + ind - 1
            
    # Second traverse to minimize boundary effect
    pointer = np.zeros([radius, n_theta])
    value[:, 1] = value[:, -1]
    
    for i in range(1, n_theta):
        for j in range(1, radius - 1):
            
            M = np.max(value[j-1:j+1, i-1])
            ind = np.argmax(value[j-1:j+1, i-1])
            
            value[j, i] = M + line_profiles[j, i]
            pointer[j,i] = j + ind - 1
    
    path = np.zeros(n_theta).astype(int)
    
    M = np.max(value[:, -1])
    ind = np.argmax(value[:, -1])
    path[-1] = ind
    
    for j in np.flip(range(1, n_theta)):
        path[j-1] = pointer[path[j], j]
        
    return path

#### Find intensity profile at a line joining two points

In [77]:
point_layer = viewer.add_points(data = None)

In [85]:
image = im_array_gad1_blur[0, :, :]

[x1, y1] = point_layer.data[0, 1:]
[x2, y2] = point_layer.data[1, 1:]
z = point_layer.data[0, 0]

profile = profile_line(image, (x1, y1), (x2, y2))
n_points = profile.shape
px_dist = np.floor(np.linalg.norm(np.array([x1, y1]) - np.array([x2, y2])))
print('Pixel distance = {0}, number of points in profile = {1}'.format(px_dist, n_points))

print('x1 = {0}, y1 = {1}, x2 = {2}, y2 = {3}'.format(x1, y1, x2, y2))

Pixel distance = 58.0, number of points in profile = (60,)
x1 = 1124.5762199044127, y1 = 10278.852064061517, x2 = 1083.8527461351187, y2 = 10320.211842108456


#### Find point where intensity falls below threshold

In [86]:
thresh = 0.1
line_profile = profile
max_val = np.max(line_profile)
max_idx = np.argmax(line_profile)
min_val_out = np.min(line_profile[max_idx:]) # Minimum intensity outside cell
thresh_val = min_val_out + thresh*(max_val - min_val_out)
r_cross = max_idx + np.where(line_profile[max_idx:] < thresh_val)[0][0]

In [87]:
plt.figure()
plt.plot(line_profile, color = 'k')

xl = np.array(plt.gca().get_xlim())
yl = np.array(plt.gca().get_ylim())
xi = np.linspace(xl[0], xl[1], 10)
yi = np.linspace(yl[0], yl[1], 10)

plt.plot(xi, np.ones(10)*max_val, color = 'r', linestyle = '--')
plt.plot(xi, np.ones(10)*min_val_out, color = 'r', linestyle = '--')
plt.plot(xi, np.ones(10)*thresh_val, color = 'b', linestyle = '--')
plt.plot(np.ones(10)*r_cross, yi, color = 'b', linestyle = '--')

#### Find intensity profile at several angles

In [92]:
center = [x1, y1]
cell_radius_px = 70
n_theta = 30
thresh = 0.2

full_radius = round(1.3*cell_radius_px)
thetas = np.linspace(0, 2*np.pi, n_theta)
[h, w] = image.shape
[x, y] = center


line_profiles = np.zeros([full_radius, n_theta])
line_profiles_mod = np.zeros([full_radius, n_theta])

r_threshold_cross = np.zeros(n_theta)

for t in range(n_theta):
    theta = thetas[t]
    f = profile_line(image, (x, y), (x + full_radius*np.cos(theta), y + full_radius*np.sin(theta)))
    line_profiles[:, t] = f[:full_radius]
    r_threshold_cross[t] = find_first_cross(f, thresh)
    line_profiles_mod[:, t] = -1*np.abs(np.array(range(full_radius)) - r_threshold_cross[t]) + full_radius;
    
plt.figure()
plt.imshow(line_profiles_mod)
plt.xlabel('Theta/4')
plt.ylabel('Distance from center (px)')
plt.title('Pixel intensity')

Text(0.5, 1.0, 'Pixel intensity')

In [67]:
fig, ax = plt.subplots(nrows = 1, ncols = 2)
ax[0].imshow(line_profiles)
ax[1].imshow(line_profiles_mod)

In [93]:
angular_dist = np.argmax(line_profiles_mod, axis = 0)
boundary = np.zeros([n_theta, 3])
boundary[:, 0] = np.ones(n_theta)*z
boundary[:, 1] = x + np.multiply(angular_dist, np.cos(thetas))
boundary[:, 2] = y + np.multiply(angular_dist, np.sin(thetas))

#### Find boundary of cell from intensity profiles

In [39]:
radius = line_profiles.shape[0]
n_theta = line_profiles.shape[1]

pointer = np.zeros([radius, n_theta])
value = np.zeros([radius, n_theta])
value[:, 0] = line_profiles_mod[:, 0]

for i in range(1, n_theta):
    for j in range(1, radius - 1):

        M = np.max(value[j-1:j+1, i-1])
        ind = np.argmax(value[j-1:j+1, i-1])

        value[j, i] = M + line_profiles_mod[j, i]
        pointer[j,i] = j + ind - 1
        
fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize = (20, 10))
ax[0, 0].imshow(value)
ax[0, 0].set_xlabel('Theta/4')
ax[0, 0].set_ylabel('Distance from center (px)')
ax[0, 0].set_title('Value - first iteration')

ax[0, 1].imshow(pointer)
ax[0, 1].set_xlabel('Theta/4')
ax[0, 1].set_ylabel('Distance from center (px)')
ax[0, 1].set_title('Pointer - first iteration')

# Second traverse to minimize boundary effect
pointer = np.zeros([radius, n_theta])
value[:, 1] = value[:, -1]

for i in range(1, n_theta):
    for j in range(1, radius - 1):

        M = np.max(value[j-1:j+1, i-1])
        ind = np.argmax(value[j-1:j+1, i-1])

        value[j, i] = M + line_profiles[j, i]
        pointer[j,i] = j + ind - 1

ax[1, 0].imshow(value)
ax[1, 0].set_xlabel('Theta/4')
ax[1, 0].set_ylabel('Distance from center (px)')
ax[1, 0].set_title('Value - second iteration')

ax[1, 1].imshow(pointer)
ax[1, 1].set_xlabel('Theta/4')
ax[1, 1].set_ylabel('Distance from center (px)')
ax[1, 1].set_title('Pointer - second iteration')

path = np.zeros(n_theta).astype(int)
    
M = np.max(value[:, -1])
ind = np.argmax(value[:, -1])
path[-1] = ind

for j in np.flip(range(1, n_theta)):
    path[j-1] = pointer[path[j], j]
    
boundary = np.zeros([n_theta, 3])
boundary[:, 0] = np.ones(n_theta)*z
boundary[:, 1] = x + np.multiply(path, np.cos(thetas))
boundary[:, 2] = y + np.multiply(path, np.sin(thetas))

# boundary = np.zeros([2, n_theta])
# boundary[0, :] = x + np.multiply(path, np.cos(thetas))
# boundary[1, :] = y + np.multiply(path, np.sin(thetas))

boundary = boundary.astype(int)
    
ax[1, 1].plot(path, color = 'r', linestyle = '--')

# Lipofuscin exlcusion

## Load existing lipofuscin ROIs

In [6]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\S5'
save_file = 'HCR_3.11_S5_lipofuscin_rois_in_cells.pkl'

with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
    l_rois = pkl.load(f)    
    print('{0} rois'.format(len(l_rois)))

14 rois


## Save lipofuscin rois from napari viewer

In [59]:
viewer.add_shapes(data = None, name = 'Lipofuscin ROIs')

<Shapes layer 'Lipofuscin ROIs' at 0x16b16848>

In [61]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\S5'
save_file = 'HCR_3.11_S5_lipofuscin_rois_in_cells.pkl'

l_rois = viewer.layers['Lipofuscin ROIs'].data
with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
    pkl.dump(l_rois, f) 
    
n_rois = len(l_rois)
print('{0} lipofuscin rois'.format(n_rois))

14 lipofuscin rois


## Get pixels in annotated lipofuscin ROIs

In [7]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\S5'
save_file = 'HCR_3.11_S5_lipofuscin_pixels_in_cells.pkl'
    
try:
    with open('{0}{1}{2}'.format(save_loc, sep, save_file), 'rb') as f:
        data_dict = pkl.load(f)
        all_lipo_pixels = data_dict['all_lipo_pixels']
        lipo_pixels_roi = data_dict['lipo_pixels_roi']
        print('Data loaded')
        
        
except IOError:
    print('No saved data found, calculating mask pixels')
    px = pixels_in_roi.pixels_in_roi(h, w, n, l_rois)
    all_lipo_pixels = px['all_pixels']
    lipo_pixels_roi = px['pixels_roi']
    with open('{0}{1}{2}'.format(save_loc, sep, save_file), 'wb') as f:
        pkl.dump({'all_lipo_pixels': all_lipo_pixels, 'lipo_pixels_roi': lipo_pixels_roi}, f)

No saved data found, calculating mask pixels
Calculated grid coordinates, 38.0 seconds
ROI 0
ROI 1
ROI 2
ROI 3
ROI 4
ROI 5
ROI 6
ROI 7
ROI 8
ROI 9
ROI 10
ROI 11
ROI 12
ROI 13


## Filter images using difference of gaussians

### Compare different filters

In [88]:
plt.figure()

sigma_small = 2
sigma_large = 5
x = np.linspace(-sigma_large*5, sigma_large*5, 1000)
small = norm.pdf(x, scale = sigma_small)
large = norm.pdf(x, scale = sigma_large)
diff = small - large
plt.plot(x, diff, label = '(2, 5)')

sigma_small = 3
sigma_large = 10
x = np.linspace(-sigma_large*5, sigma_large*5, 1000)
small = norm.pdf(x, scale = sigma_small)
large = norm.pdf(x, scale = sigma_large)
diff = small - large
plt.plot(x, diff, label = '(3, 10)')

sigma_small = 5
sigma_large = 10
x = np.linspace(-sigma_large*5, sigma_large*5, 1000)
small = norm.pdf(x, scale = sigma_small)
large = norm.pdf(x, scale = sigma_large)
diff = small - large
plt.plot(x, diff, label = '(5, 10)')

# sigma_small = 1.5
# sigma_large = 3.75
# x = np.linspace(-sigma_large*5, sigma_large*5, 1000)
# small = norm.pdf(x, scale = sigma_small)
# large = norm.pdf(x, scale = sigma_large)
# diff = small - large
# plt.plot(x, diff, label = '(1.5, 3.75)')

# sigma_small = 1
# sigma_large = 2
# x = np.linspace(-sigma_large*5, sigma_large*5, 1000)
# small = norm.pdf(x, scale = sigma_small)
# large = norm.pdf(x, scale = sigma_large)
# diff = small - large
# plt.plot(x, diff, label = '(1, 2)')

sigma_small = 10
sigma_large = 15
x = np.linspace(-sigma_large*5, sigma_large*5, 1000)
small = norm.pdf(x, scale = sigma_small)
large = norm.pdf(x, scale = sigma_large)
diff = small - large
plt.plot(x, diff, label = '(10, 15)')

sigma_small = 7
sigma_large = 10
x = np.linspace(-sigma_large*5, sigma_large*5, 1000)
small = norm.pdf(x, scale = sigma_small)
large = norm.pdf(x, scale = sigma_large)
diff = small - large
plt.plot(x, diff, label = '(7, 10)')

plt.legend()

### Diff_gauss2

In [9]:
from scipy.stats import norm
from skimage.filters import gaussian

def diff_gauss2(sigma_small, sigma_large, img, do_plot = 1):

    #sigma_large = 10 # In pixels
    #sigma_small = 3 # In pixels

    #x = np.linspace(norm.ppf(0.01), norm.ppf(0.99), 1000)
    x = np.linspace(-sigma_large*5, sigma_large*5, 1000)
    small = norm.pdf(x, scale = sigma_small)
    large = norm.pdf(x, scale = sigma_large)
    diff = small - large

    n = img.shape[0]
    im_diff = np.zeros(img.shape)

    for plane in range(n):
        im_filt_large = gaussian(img[plane, :, :], sigma = sigma_large)
        im_filt_small = gaussian(img[plane, :, :], sigma = sigma_small)
        im_diff[plane, :, :] = im_filt_small - im_filt_large

    if do_plot:

        plt.figure()
        plt.plot(x, diff, label = 'diff')
        plt.plot(x, small, label = 'Small sigma filter')
        plt.plot(x, large, label = 'Large sigma filter')
        plt.xlabel('Distance from center of kernel (px)')
        plt.ylabel('Kernel')
        plt.title('Difference of gaussians filter')
        plt.legend()

    return im_diff


### Do filtering

#### Diff_gauss2

In [10]:
t0 = time.time()    

print('Filtering images (~5 minutes)')
sigma_small = 5
sigma_large = 10


img = im_array_ndnf
im_diff_ndnf_lipo = diff_gauss2(sigma_small, sigma_large, img, do_plot = 1)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

img = im_array_sst
im_diff_sst_lipo = diff_gauss2(sigma_small, sigma_large, img, do_plot = 0)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

img = im_array_vip
im_diff_vip_lipo = diff_gauss2(sigma_small, sigma_large, img, do_plot = 0)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

# img = im_array_gad1
# im_diff_gad1_lipo = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 1)
# t1 = time.time() - t0
# print('{0} seconds'.format(int(t1)))

del img

Filtering images (~5 minutes)
85 seconds
169 seconds
272 seconds


#### Simple gaussian filter

In [17]:
t0 = time.time()    

print('Filtering images (~5 minutes)')
sigma = 3


img = im_array_ndnf
im_ndnf_lipo = gaussian(img, sigma)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

img = im_array_sst
im_sst_lipo = gaussian(img, sigma)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

img = im_array_vip
im_vip_lipo = gaussian(img, sigma)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

# img = im_array_gad1
# im_diff_gad1_lipo = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 1)
# t1 = time.time() - t0
# print('{0} seconds'.format(int(t1)))

del img

Filtering images (~5 minutes)
7 seconds
14 seconds
21 seconds


### Save filtered images using h5py

In [55]:
t0 = time.time()
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'

save_file = 'Gad1_im_diff_sigma_small = {0}, sigma_large = {1}'.format(sigma_small, sigma_large)
h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file), 'w')
h5f.create_dataset('dataset_1', data=im_diff_gad1)
h5f.close()

print('im_diff_gad1 saved: {0} seconds'.format(np.round(time.time() - t0)))
    
save_file = 'Ndnf_im_diff_sigma_small = {0}, sigma_large = {1}'.format(sigma_small, sigma_large)
h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file), 'w')
h5f.create_dataset('dataset_1', data=im_diff_ndnf)
h5f.close()

print('im_diff_ndnf saved: {0} seconds'.format(np.round(time.time() - t0)))
    
save_file = 'Sst_im_diff_sigma_small = {0}, sigma_large = {1}'.format(sigma_small, sigma_large)
h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file), 'w')
h5f.create_dataset('dataset_1', data=im_diff_sst)
h5f.close()

print('im_diff_sst saved: {0} seconds'.format(np.round(time.time() - t0)))
    
save_file = 'Vip_im_diff_sigma_small = {0}, sigma_large = {1}'.format(sigma_small, sigma_large)
h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file), 'w')
h5f.create_dataset('dataset_1', data=im_diff_vip)
h5f.close()

print('im_diff_vip saved: {0} seconds'.format(np.round(time.time() - t0)))

im_diff_gad1 saved: 13.0 seconds
im_diff_ndnf saved: 27.0 seconds
im_diff_sst saved: 41.0 seconds
im_diff_vip saved: 57.0 seconds


## Binarize filtered images and plot threshold

In [12]:
img_dict = {'Ndnf': im_diff_ndnf_lipo, 'Vip': im_diff_vip_lipo, 'Sst': im_diff_sst_lipo}
# img_dict = {'Ndnf': im_array_ndnf, 'Sst': im_array_sst, 'Vip': im_array_vip}
#img_dict = {'Ndnf': im_ndnf_lipo, 'Sst': im_sst_lipo, 'Vip': im_vip_lipo}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}
#pixels_dict = {'Cells': all_cell_pixels}

colors_dict = {'Cells': 'b', 'Lipofuscin': 'k'}

title = 'Diff gauss sigma = ({0}, {1})'.format(sigma_small, sigma_large)
#title = 'Raw images'
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5\\Plots'
save_file = '{0} hist lipo + cells.png'.format(title)
#save_file = '{0} hist cells.png'.format(title)

thresh_scale = 1

thresh = hist_channels.hist_channels(img_dict, pixels_dict, max_per_group= 100000, 
                                     do_bin = True, bin_group = 'Cells', thresh_scale = thresh_scale,
                                     colors_dict = colors_dict, 
                                     title = title, save = True, save_loc = save_loc, save_file = save_file)

del img_dict
del pixels_dict

im_bin_ndnf_lipo = np.zeros(im_diff_ndnf_lipo.shape)
im_bin_sst_lipo = np.zeros(im_diff_ndnf_lipo.shape)
im_bin_vip_lipo = np.zeros(im_diff_ndnf_lipo.shape)
#im_bin_gad1 = np.zeros(im_diff_gad1_lipo.shape)

ix_ndnf = np.where(im_diff_ndnf_lipo > thresh['Ndnf'])
ix_sst = np.where(im_diff_sst_lipo > thresh['Sst'])
ix_vip = np.where(im_diff_vip_lipo > thresh['Vip'])
# ix_ndnf = np.where(im_ndnf_lipo > thresh['Ndnf'])
# ix_sst = np.where(im_sst_lipo > thresh['Sst'])
# ix_vip = np.where(im_vip_lipo > thresh['Vip'])

# ix_gad1 = np.where(im_diff_gad1_lipo > thresh['Gad1'])

im_bin_ndnf_lipo[ix_ndnf] = im_diff_ndnf_lipo[ix_ndnf]
im_bin_sst_lipo[ix_sst] = im_diff_sst_lipo[ix_sst]
im_bin_vip_lipo[ix_vip] = im_diff_vip_lipo[ix_vip]
# im_bin_ndnf_lipo[ix_ndnf] = im_ndnf_lipo[ix_ndnf]
# im_bin_sst_lipo[ix_sst] = im_sst_lipo[ix_sst]
# im_bin_vip_lipo[ix_vip] = im_vip_lipo[ix_vip]



#im_bin_gad1_lipo[ix_gad1] = im_diff_gad1_lipo[ix_gad1]

Finding threshold for binarizing
Finding threshold for binarizing
Finding threshold for binarizing


In [80]:
plt.close('all')

In [63]:
del im_array_lipo

In [61]:
del im_diff_ndnf
del im_diff_vip
del im_diff_sst

In [60]:
del im_bin_ndnf_lipo

NameError: name 'im_bin_ndnf_lipo' is not defined

In [58]:
del im_bin_vip_lipo
del im_bin_sst_lipo

In [255]:
im_bin_gad1_lipo = im_bin_gad1
del im_bin_gad1

In [242]:
plt.close('all')

## Select lipofuscin cloud

### Plot lipofuscin pixels in 2D scatter (2 channels)

In [22]:
fig = plt.figure()
x_vals = [im_ndnf_lipo[a[0], a[1], a[2]] for a in all_lipo_pixels]
y_vals = [im_vip_lipo[a[0], a[1], a[2]] for a in all_lipo_pixels]

plt.scatter(x_vals, y_vals)
roi = roipoly()

In [23]:
t0 = time.time()
mask = roi.getMask()[0]
t1 = time.time() - t0
print(np.round(t1))

56.0


### Draw polygon to select lipofuscin cloud

In [25]:
n_lipo_px = len(all_lipo_pixels)
xi = np.zeros(n_lipo_px, dtype = int)
yi = np.zeros(n_lipo_px, dtype = int)

ix = np.zeros(n_lipo_px, dtype = bool)
x0, x1 = fig.axes[0].get_xlim()
y0, y1 = fig.axes[0].get_ylim()
x = np.linspace(x0, x1, mask.shape[1])
y = np.linspace(y0, y1, mask.shape[0])

for i in range(n_lipo_px):
    if np.mod(i, 10) == 0:
        print(i)
    px = all_lipo_pixels[i]
    xi[i] = bisect_left(x, im_ndnf_lipo[px[0], px[1], px[2]])
    yi[i] = bisect_left(y, im_vip_lipo[px[0], px[1], px[2]])
    if mask[yi[i], xi[i]] > 0:
        ix[i] = True

all_lipo_pixels = all_lipo_pixels[ix]

x_vals = [im_ndnf_lipo[a[0], a[1], a[2]] for a in all_lipo_pixels]
y_vals = [im_vip_lipo[a[0], a[1], a[2]] for a in all_lipo_pixels]

plt.scatter(x_vals, y_vals, color = 'r')

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910


#### Roipoly

In [21]:
# This code is from https://github.com/jdoepfert, 
# which can be used without permission
import matplotlib.path as mplPath

class roipoly:
    def __init__(self, fig=[], ax=[], roicolor='b'):
        if fig == []:
            fig = plt.gcf()

        if ax == []:
            ax = plt.gca()

        self.previous_point = []
        self.allxpoints = []
        self.allypoints = []
        self.start_point = []
        self.end_point = []
        self.line = None
        self.roicolor = roicolor
        self.fig = fig
        self.ax = ax
        self.mask = []

        self.__ID1 = self.fig.canvas.mpl_connect(
            'motion_notify_event', self.__motion_notify_callback)
        self.__ID2 = self.fig.canvas.mpl_connect(
            'button_press_event', self.__button_press_callback)
#         self.__ID3 = self.fig.canvas.mpl_connect(
#             'key_press_event', self.__key_press_callback)

    def getMask(self):
        
        ny = 20000
        nx = 20000
        poly_verts = [(self.allxpoints[0], self.allypoints[0])]
        for i in range(len(self.allxpoints)-1, -1, -1):
            poly_verts.append((self.allxpoints[i], self.allypoints[i]))

        # Create vertex coordinates for each grid cell...
        # (<0,0> is at the top left of the grid in this system)
        x0, x1 = self.ax.get_xlim()
        y0, y1 = self.ax.get_ylim()
        x, y = np.meshgrid(np.linspace(x0, x1, nx), np.linspace(y0, y1, ny))
        x, y = x.flatten(), y.flatten()
        points = np.vstack((x,y)).T

        ROIpath = mplPath.Path(poly_verts)
        grid = ROIpath.contains_points(points).reshape((ny,nx)), 0
        return grid

    def __motion_notify_callback(self, event):
        if event.inaxes:
            ax = event.inaxes
            x, y = event.xdata, event.ydata
            # Move line around
            if (event.button == None or event.button == 1) and self.line != None: 
                self.line.set_data([self.previous_point[0], x],
                                   [self.previous_point[1], y])
                self.fig.canvas.draw()


#     def __key_press_callback(self, event):
            
        
    def __button_press_callback(self, event):
        if event.inaxes:
            x, y = event.xdata, event.ydata
            ax = event.inaxes
            # If you press the left button, single click
            if event.button == 1 and event.dblclick == False:  
                if self.line == None: # if there is no line, create a line
                    self.line = plt.Line2D([x, x],
                                           [y, y],
                                           marker='o',
                                           color=self.roicolor)
                    self.start_point = [x,y]
                    self.previous_point =  self.start_point
                    self.allxpoints=[x]
                    self.allypoints=[y]

                    ax.add_line(self.line)
                    self.fig.canvas.draw()
                    # add a segment
                else: # if there is a line, create a segment
                    self.line = plt.Line2D([self.previous_point[0], x],
                                           [self.previous_point[1], y],
                                           marker = 'o',
                                           color=self.roicolor)
                    self.previous_point = [x,y]
                    self.allxpoints.append(x)
                    self.allypoints.append(y)

                    event.inaxes.add_line(self.line)
                    self.fig.canvas.draw()
            elif ((event.button == 1 and event.dblclick==True) or (event.button == 3 and event.dblclick==False)) and self.line != None: # close the loop and disconnect
                    
                    self.fig.canvas.mpl_disconnect(self.__ID1) #joerg
                    self.fig.canvas.mpl_disconnect(self.__ID2) #joerg

                    self.line.set_data([self.previous_point[0],
                                    self.start_point[0]],
                                   [self.previous_point[1],
                                    self.start_point[1]])
                    self.last_line = ax.add_line(self.line)
                    self.fig.canvas.draw()
                    self.line = None

                    if sys.flags.interactive:
                        pass
                    


## Find mahalanobis distance of pixels from lipofuscin cloud

In [190]:
np.exp(2)

7.38905609893065

In [39]:
#img_dict = {'Ndnf': im_bin_ndnf_lipo, 'Vip': im_bin_vip_lipo}
img_dict = {'Ndnf': im_bin_ndnf_lipo, 'Sst': im_bin_sst_lipo, 'Vip': im_bin_vip_lipo}
#img_dict = {'Ndnf': im_diff_ndnf, 'Sst': im_diff_sst, 'Vip': im_diff_vip}
#img_dict = {'Ndnf': im_array_ndnf, 'Sst': im_array_sst, 'Vip': im_array_vip}
#img_dict = {'Ndnf': im_ndnf_lipo, 'Sst': im_sst_lipo, 'Vip': im_vip_lipo}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}

origin_group = 'Lipofuscin'
bin_group = 'Cells'

colors_dict = {'Cells': 'b', 'Lipofuscin': 'k'}

#title = 'Diff gauss sigma = ({0}, {1}); binarized above {2} std from median'.format(sigma_small, sigma_large, thresh_scale)
#title = 'Raw images'
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5\\Plots'
#save_file = '{0} hist lipo + cells.png'.format(title)
save_file = 'Gauss blur sigma = 3px'
#save_file = '{0} hist cells.png'.format(title)

output = m_dist.m_dist(img_dict, pixels_dict, origin_group, bin_group, colors_dict = colors_dict, thresh_scale = 0,
#                             title = title, 
                       save = True, save_loc = save_loc, save_file = save_file)

m_dist_vals = output['m_dist']
thresh = output['thresh']
cov = output['cov']

print('Threshold log mahalanobis distance = {0}'.format(np.log(thresh)))

del img_dict
del pixels_dict

Threshold log mahalanobis distance = 0.7522689464693758


##### Not necessary to run this cell - it's a part of m_dist.py

In [37]:

channel_names = list(img_dict.keys())
n_channels = len(channel_names)

px_groups = list(pixels_dict.keys())


# %% Get values for each channel for each pixel group
px_vals = {}

for group in px_groups:

    px = pixels_dict[group]


    px_vals[group] = {}

    for channel in channel_names:
        px_vals[group][channel] = img_dict[channel][(px[:, 0], px[:, 1], px[:, 2])]


# %% Get mean and covariance matrix of origin group
mu = np.zeros(n_channels)
origin_px = np.zeros([n_channels, px_vals[origin_group][channel_names[0]].shape[0]])
for c in range(n_channels):
    mu[c] = np.mean(px_vals[origin_group][channel_names[c]])
    origin_px[c, :] = px_vals[origin_group][channel_names[c]]

cov = np.cov(origin_px) 
VI = np.linalg.inv(cov)     

In [196]:
all_points = m_dist_vals['Cells']
center = np.median(all_points)
neg_points = all_points[all_points < center]
pos_points = neg_points + center
all_points = np.concatenate([neg_points, pos_points])
s = np.std(all_points)

In [197]:
s

1.3070425864249915

In [198]:
(np.exp(2) - center)/s

3.712910596417829

In [200]:
np.log(thresh)

1.997713646069735

In [41]:
plt.figure()
plt.imshow(np.log(cov))
plt.colorbar(label = 'Log covariance')

## Use mahalanobis distance to label all lipofuscin pixels 

In [32]:
lipo_pixels = all_cell_pixels[m_dist_vals['Cells'] < thresh, :]

## Don't exclude pixels below some average intensity

In [33]:
lipo_vals = np.zeros([3, len(lipo_pixels)])
lipo_vals[0, :] = np.array([im_bin_ndnf_lipo[a[0], a[1], a[2]] for a in lipo_pixels])
lipo_vals[1, :] = np.array([im_bin_vip_lipo[a[0], a[1], a[2]] for a in lipo_pixels])
lipo_vals[2, :] = np.array([im_bin_sst_lipo[a[0], a[1], a[2]] for a in lipo_pixels])

mean_vals = np.mean(lipo_vals, axis = 0)
print(mean_vals.shape)

lipo_low_cutoff = np.exp(5)
lipo_pixels = lipo_pixels[mean_vals > lipo_low_cutoff] 

im_array_lipo = np.zeros(im_array_ndnf.shape)
im_array_lipo[lipo_pixels[:, 0], lipo_pixels[:, 1], lipo_pixels[:, 2]] = np.ones(lipo_pixels.shape[0])


(757295,)


## Re-calculate cell masks excluding lipofuscin pixels

In [34]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\S5'
save_file = 'HCR_3.11_S5_cell_masks_without_lipofuscin.pkl'

try:
    with open('{0}{1}{2}'.format(save_loc, sep, save_file), 'rb') as f:
        cell_pixels_no_lipo = pkl.load(f)
        print('Data loaded')
        jhk
except:
    
    print('Calculating')
    cell_pixels_no_lipo = {}

    cells = mask_vertices.keys()
#     cells = cells_sub + 1
    n_px_total = 0
    for cell in cells:

        data = Cell.cell_data[cell]
        planes = data['z_planes']
        cell_pixels_no_lipo[cell] = {}

        for plane in planes:

            x = mask_vertices[cell][plane][0]
            y = mask_vertices[cell][plane][1]
            n = len(x)
            cell_pixels_no_lipo[cell][plane] = np.array([[x[i], y[i]] for i in range(n) if im_array_lipo[plane, x[i], y[i]] == 0])
            cell_pixels_no_lipo[cell][plane] = np.transpose(cell_pixels_no_lipo[cell][plane])
#             cell_pixels_no_lipo[cell][plane] = {}
#             n_px = mask_vertices[cell][plane][0].shape[0]
#             m = m_dist_vals['Cells'][n_px_total:n_px_total + n_px] # Mahalanobis distance values for pixels in cell mask
#             px_coords = np.array(mask_vertices[cell][plane])
#             cell_pixels_no_lipo[cell][plane] = px_coords[:, m >= thresh]
#             #cell_pixels_no_lipo[cell][plane] = px_coords[:, m ]
#             n_px_total += n_px
            


    if not n_px_total == all_cell_pixels.shape[0]:
        print('Total number of pixels incorrect, could have removed incorrect pixels for cell')\

    with open('{0}{1}{2}'.format(save_loc, sep, save_file), 'wb') as f:
        pkl.dump(cell_pixels_no_lipo, f)

Data loaded
Calculating
Total number of pixels incorrect, could have removed incorrect pixels for cell


In [180]:
cells

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        28,  30,  31,  32,  33,  34,  35,  37,  38,  39,  40,  41,  42,
        43,  44,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  78,  80,  81,  82,  83,  84,  85,
        86,  87,  88,  89,  90,  91,  92,  93,  95,  96,  97,  99, 102,
       104, 105, 106, 110, 111, 120, 125, 132, 140, 141, 143, 145, 146,
       165])

In [177]:
cell_pixels_no_lipo.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 45, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 96, 97, 98, 100, 103, 105, 106, 107, 111, 112, 121, 126, 133, 141, 142, 144, 146, 147, 166])

## Delete large arrays

In [ ]:
del im_diff_ndnf
del im_diff_sst
del im_diff_vip
del im_bin_ndnf
del im_bin_vip
del im_bin_sst


In [ ]:
del im_bin_gad1
del im_diff_gad1

In [ ]:
del im_array_ndnf
del im_array_sst
del im_array_vip
del im_array_gad1
del im_array_dapi

# Classify cells by gene expression

## Calculate intensity for each gene in each cell 

### Average intensity in all pixels in cell mask

In [40]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'
save_file = 'S5_data_ndnf_sst_vip.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        sig = pkl.load(f)
    avg_intensity_ndnf = sig['Ndnf']
    avg_intensity_sst = sig['Sst'] 
    avg_intensity_vip = sig['Vip']
    print('Data loaded')

except:
    print('Data not found, calculating intensities')
    no_cells = len(Cell.cell_data.keys())
    avg_intensity_ndnf = np.zeros(no_cells)
    avg_intensity_vip = np.zeros(no_cells)
    avg_intensity_sst = np.zeros(no_cells)

    for cell in range(no_cells):

        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        intensity_ndnf = np.zeros(cell_dict['no_planes'])
        intensity_vip = np.zeros(cell_dict['no_planes'])
        intensity_sst = np.zeros(cell_dict['no_planes'])

        i = 0
        for plane in z_planes:

            xy_coord = mask_vertices[cell_no][plane]
            x = xy_coord[0]
            y = xy_coord[1]
            intensity_ndnf[i] = np.mean(im_array_ndnf[plane, x, y])
            intensity_vip[i] = np.mean(im_array_vip[plane, x, y])
            intensity_sst[i] = np.mean(im_array_sst[plane, x, y])
            i += 1

        avg_intensity_ndnf[cell] = np.mean(intensity_ndnf)
        avg_intensity_vip[cell] = np.mean(intensity_vip)
        avg_intensity_sst[cell] = np.mean(intensity_sst)

        with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
            pkl.dump({'Ndnf': avg_intensity_ndnf, 'Sst': avg_intensity_sst, 'Vip':avg_intensity_vip}, f)


Data loaded


### Intensity in masks excluding lipofuscin pixels

In [46]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'
save_file = 'S5_data_ndnf_sst_vip.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        sig = pkl.load(f)
    int_ndnf = sig['Ndnf_lipo_excl']
    int_sst = sig['Sst_lipo_excl'] 
    int_vip = sig['Vip_lipo_excl']
    print('Data loaded')
    ejkl

except:
    print('Data not found, calculating intensities')
    no_cells = len(Cell.cell_data.keys())
    int_ndnf = np.zeros(no_cells)
    int_vip = np.zeros(no_cells)
    int_sst = np.zeros(no_cells)

    for cell in range(no_cells):
    #for cell in cells_sub:

        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        intensity_ndnf = np.zeros(cell_dict['no_planes'])
        intensity_vip = np.zeros(cell_dict['no_planes'])
        intensity_sst = np.zeros(cell_dict['no_planes'])

        i = 0
        for plane in z_planes:

            xy_coord = cell_pixels_no_lipo[cell_no][plane]
            
            if xy_coord.shape[1] == 0: # all pixels are lipofuscin pixels
                intensity_ndnf[i] = 0
                intensity_vip[i] = 0
                intensity_sst[i] = 0
                continue
                
            x = xy_coord[0, :]
            y = xy_coord[1, :]
            intensity_ndnf[i] = np.mean(im_array_ndnf[plane, x, y])
            intensity_vip[i] = np.mean(im_array_vip[plane, x, y])
            intensity_sst[i] = np.mean(im_array_sst[plane, x, y])
            i += 1

        int_ndnf[cell] = np.mean(intensity_ndnf)
        int_vip[cell] = np.mean(intensity_vip)
        int_sst[cell] = np.mean(intensity_sst)
        
        sig['Ndnf_lipo_excl'] = int_ndnf
        sig['Sst_lipo_excl'] = int_sst 
        sig['Vip_lipo_excl'] = int_vip 

        with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
            pkl.dump(sig, f)


Data loaded
Data not found, calculating intensities
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280
Cell 290
Cell 300
Cell 310
Cell 320
Cell 330
Cell 340
Cell 350


### Intensity in masks excluding lipofuscin pixels in binarized images (essentially spot counting)

#### Filter images (smaller filter than for finding lipofuscin pixels)

In [42]:
t0 = time.time()    

print('No images found, filtering images (~5 minutes)')
sigma_small = 1
sigma_large = 2


img = im_array_ndnf
im_diff_ndnf = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 1)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

img = im_array_sst
im_diff_sst = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 0)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

img = im_array_vip
im_diff_vip = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 0)
t1 = time.time() - t0
print('{0} seconds'.format(int(t1)))

#     img = im_array_gad1
#     im_diff_gad1 = diff_gauss.diff_gauss(sigma_small, sigma_large, img, do_plot = 1)
#     t1 = time.time() - t0
#     print('{0} seconds'.format(int(t1)))

del img

No images found, filtering images (~5 minutes)
147 seconds
283 seconds
537 seconds


##### Save filtered images

In [ ]:
t0 = time.time()
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'

save_file = 'Gad1_im_diff_sigma_small = {0}, sigma_large = {1}'.format(sigma_small, sigma_large)
h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file), 'w')
h5f.create_dataset('dataset_1', data=im_diff_gad1)
h5f.close()

print('im_diff_gad1 saved: {0} seconds'.format(np.round(time.time() - t0)))
    
save_file = 'Ndnf_im_diff_sigma_small = {0}, sigma_large = {1}'.format(sigma_small, sigma_large)
h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file), 'w')
h5f.create_dataset('dataset_1', data=im_diff_ndnf)
h5f.close()

print('im_diff_ndnf saved: {0} seconds'.format(np.round(time.time() - t0)))
    
save_file = 'Sst_im_diff_sigma_small = {0}, sigma_large = {1}'.format(sigma_small, sigma_large)
h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file), 'w')
h5f.create_dataset('dataset_1', data=im_diff_sst)
h5f.close()

print('im_diff_sst saved: {0} seconds'.format(np.round(time.time() - t0)))
    
save_file = 'Vip_im_diff_sigma_small = {0}, sigma_large = {1}'.format(sigma_small, sigma_large)
h5f = h5py.File('{0}{1}{2}.h5'.format(save_loc, sep, save_file), 'w')
h5f.create_dataset('dataset_1', data=im_diff_vip)
h5f.close()

print('im_diff_vip saved: {0} seconds'.format(np.round(time.time() - t0)))

#### Binarize

In [59]:
img_dict = {'Ndnf': im_diff_ndnf, 'Sst': im_diff_sst, 'Vip': im_diff_vip}
#img_dict = {'Ndnf': im_array_ndnf, 'Sst': im_array_sst, 'Vip': im_array_vip}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}
#pixels_dict = {'Cells': all_cell_pixels}

colors_dict = {'Cells': 'b', 'Lipofuscin': 'k'}

title = 'Diff gauss sigma = ({0}, {1})'.format(sigma_small, sigma_large)
#title = 'Raw images'
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5\\Plots'
save_file = '{0} hist lipo + cells.png'.format(title)
#save_file = '{0} hist cells.png'.format(title)

thresh_scale = 12

thresh = hist_channels.hist_channels(img_dict, pixels_dict, max_per_group= 100000, 
                                     do_bin = True, bin_group = 'Cells', thresh_scale = thresh_scale,
                                     colors_dict = colors_dict, 
                                     title = title, save = True, save_loc = save_loc, save_file = save_file)

del img_dict
del pixels_dict

im_bin_ndnf = np.zeros(im_diff_ndnf.shape)
im_bin_sst = np.zeros(im_diff_ndnf.shape)
im_bin_vip = np.zeros(im_diff_ndnf.shape)
#im_bin_gad1 = np.zeros(im_diff_gad1.shape)

ix_ndnf = np.where(im_diff_ndnf > thresh['Ndnf'])
ix_sst = np.where(im_diff_sst > thresh['Sst'])
ix_vip = np.where(im_diff_vip > thresh['Vip'])
#ix_gad1 = np.where(im_diff_gad1 > thresh['Gad1'])

im_bin_ndnf[ix_ndnf] = im_diff_ndnf[ix_ndnf]
im_bin_sst[ix_sst] = im_diff_sst[ix_sst]
im_bin_vip[ix_vip] = im_diff_vip[ix_vip]
#im_bin_gad1[ix_gad1] = im_diff_gad1[ix_gad1]

Finding threshold for binarizing
Finding threshold for binarizing
Finding threshold for binarizing


#### Calculate intensities

In [60]:
# Note: size of spot is also relevant here. 
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'
save_file = 'S5_data_ndnf_sst_vip.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        sig = pkl.load(f)
    int_ndnf_bin = sig['Ndnf_lipo_excl_bin']
    int_sst_bin = sig['Sst_lipo_excl_bin'] 
    int_vip_bin = sig['Vip_lipo_excl_bin']
    print('Data loaded')
    jfio

except:
    print('Data not found, calculating intensities')
    no_cells = len(Cell.cell_data.keys())
    int_ndnf_bin = np.zeros(no_cells)
    int_vip_bin = np.zeros(no_cells)
    int_sst_bin = np.zeros(no_cells)

    for cell in range(no_cells):
#     for cell in cells_sub:
        
        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        intensity_ndnf = np.zeros(cell_dict['no_planes'])
        intensity_vip = np.zeros(cell_dict['no_planes'])
        intensity_sst = np.zeros(cell_dict['no_planes'])

        i = 0
        for plane in z_planes:

            xy_coord = cell_pixels_no_lipo[cell_no][plane]
            
            if xy_coord.shape[1] == 0: # all pixels are lipofuscin pixels
                intensity_ndnf[i] = 0
                intensity_vip[i] = 0
                intensity_sst[i] = 0
                continue
                
            x = xy_coord[0, :]
            y = xy_coord[1, :]
            intensity_ndnf[i] = np.mean(im_bin_ndnf[plane, x, y])
            intensity_vip[i] = np.mean(im_bin_vip[plane, x, y])
            intensity_sst[i] = np.mean(im_bin_sst[plane, x, y])
            i += 1

        int_ndnf_bin[cell] = np.mean(intensity_ndnf)
        int_vip_bin[cell] = np.mean(intensity_vip)
        int_sst_bin[cell] = np.mean(intensity_sst)
        
    sig['Ndnf_lipo_excl_bin'] = int_ndnf_bin
    sig['Sst_lipo_excl_bin'] = int_sst_bin 
    sig['Vip_lipo_excl_bin'] = int_vip_bin 

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(sig, f)



Data loaded
Data not found, calculating intensities
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280
Cell 290
Cell 300
Cell 310
Cell 320
Cell 330
Cell 340
Cell 350


### Intensity in masks excluding lipofuscin pixels in diff images

In [23]:
# Note: size of spot is also relevant here. 

save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'
save_file = 'S5_data2_ndnf_sst_vip.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        sig = pkl.load(f)
    int_ndnf_diff = sig['ndnf_lipo_excl_diff']
    int_sst_diff = sig['sst_lipo_excl_diff'] 
    int_vip_diff = sig['vip_lipo_excl_diff']
    print('Data loaded')

except:
    print('Data not found, calculating intensities')
    no_cells = len(Cell.cell_data.keys())
    int_ndnf_diff = np.zeros(no_cells)
    int_vip_diff = np.zeros(no_cells)
    int_sst_diff = np.zeros(no_cells)

    for cell in range(no_cells):

        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        intensity_ndnf = np.zeros(cell_dict['no_planes'])
        intensity_vip = np.zeros(cell_dict['no_planes'])
        intensity_sst = np.zeros(cell_dict['no_planes'])

        i = 0
        for plane in np.array(z_planes).astype(int):

            xy_coord = cell_pixels_no_lipo[cell_no][plane]
            x = xy_coord[0]
            y = xy_coord[1]
            intensity_ndnf[i] = np.mean(im_diff_ndnf[plane, x, y])
            intensity_vip[i] = np.mean(im_diff_vip[plane, x, y])
            intensity_sst[i] = np.mean(im_diff_sst[plane, x, y])
            i += 1

        int_ndnf_diff[cell] = np.mean(intensity_ndnf)
        int_vip_diff[cell] = np.mean(intensity_vip)
        int_sst_diff[cell] = np.mean(intensity_sst)
        
        sig['ndnf_lipo_excl_bin'] = int_ndnf_diff
        sig['sst_lipo_excl_bin'] = int_sst_diff
        sig['vip_lipo_excl_bin'] = int_vip_diff

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
            pkl.dump(sig, f)


Data not found, calculating intensities
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280
Cell 290
Cell 300
Cell 310
Cell 320
Cell 330
Cell 340
Cell 350


## Local background

### Raw images

In [53]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'
save_file = 'S5_background_ndnf_sst_vip.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        dict = pkl.load(f)
    
    avg_bg_ndnf = dict['Ndnf'] 
    avg_bg_sst = dict['Sst']  
    avg_bg_vip = dict['Vip']
    centers = dict['Centers']
    n_vert_all = dict['n_vert_all']
    #bg_all = dict['bg_all']
    ghj
    print('Data loaded')

except:
    
    print('Data not found, calculating background values')
    bg_all = np.ones([n, h, w])
    n_vert_all = {}
    centers = {}

    no_cells = len(Cell.cell_data.keys())
    for cell in range(no_cells):

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']
        n_vert_all[cell] = {}
        centers[cell] = {}

        for plane in z_planes:

            xy_coord = mask_vertices[cell_no][plane]
            x = xy_coord[0]
            y = xy_coord[1]
            z = plane
            centers[cell][plane] = np.array([np.mean(x), np.mean(y)]).astype(int)


            bg_all[z, x, y] = np.zeros([1, len(x)])

            n_vert_all[cell][plane] = len(x) 
            
    # fig, ax = plt.subplots(nrows = 3, ncols = 20)
    um_per_px = 0.13
    no_cells = len(Cell.cell_data.keys())
    avg_bg_ndnf = np.zeros(no_cells)
    avg_bg_vip = np.zeros(no_cells)
    avg_bg_sst = np.zeros(no_cells)

    local_region = 50 # in um
    local_px = int(local_region/um_per_px/2)

    min_dist = 10 # in um
    min_dist_px = min_dist/um_per_px

    for cell in range(no_cells):
        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bg_ndnf = np.zeros(len(z_planes))
        bg_vip = np.zeros(len(z_planes))
        bg_sst = np.zeros(len(z_planes))

        p = 0
        for plane in z_planes:

            center = centers[cell][plane]
            n_verts = n_vert_all[cell][plane]

            x1 = np.max([0, center[0] - local_px])
            x2 = np.min([h, center[0] + local_px])
            y1 = np.max([0, center[1] - local_px])
            y2 = np.min([w, center[1] + local_px])

            center_local = [local_px, local_px]

            local_bg = bg_all[plane, x1:x2, y1:y2].astype(bool)
            local_im_ndnf = im_array_ndnf[plane, x1:x2, y1:y2]
            local_im_sst = im_array_sst[plane, x1:x2, y1:y2]
            local_im_vip = im_array_vip[plane, x1:x2, y1:y2]

    #         ax[0, cell].imshow(local_im, vmin = np.min(local_im), vmax = np.max(local_im)/2)

    #         ax[1, cell].imshow(local_bg)

            verts = np.array(np.where(local_bg))
            dist_from_center = np.linalg.norm(verts - np.reshape(center_local, [2, 1]), axis = 0)
            verts = verts[:, dist_from_center > min_dist_px]
            dist_from_center = dist_from_center[dist_from_center > min_dist_px]

            order = np.argsort(dist_from_center)
            n_verts = np.min([n_verts, len(order)])

            x_bg = verts[0][order[0:n_verts]]
            y_bg = verts[1][order[0:n_verts]]

            local_bg = np.zeros(local_bg.shape).astype(bool)
            local_bg[x_bg, y_bg] = np.ones(n_verts)

    #         ax[2, cell].imshow(local_bg)

            bg_ndnf[p] = np.mean(local_im_ndnf[local_bg])
            bg_vip[p] = np.mean(local_im_vip[local_bg])
            bg_sst[p] = np.mean(local_im_sst[local_bg])

            p += 1



        avg_bg_ndnf[cell] = np.mean(bg_ndnf)
        avg_bg_vip[cell] = np.mean(bg_vip)
        avg_bg_sst[cell] = np.mean(bg_sst)

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump({'Ndnf': avg_bg_ndnf, 'Sst': avg_bg_sst, 'Vip':avg_bg_vip, 'Centers': centers, 
                  'n_vert_all':n_vert_all, 
                  #'bg_all': bg_all
                 }, f)
            
    #del bg_all

Data not found, calculating background values


### Binarized images

In [26]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'
save_file = 'S5_background_ndnf_sst_vip.pkl'

with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        bg_dict = pkl.load(f)  

In [61]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'
save_file = 'S5_background_ndnf_sst_vip.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        bg_dict = pkl.load(f)   
    
    centers = bg_dict['Centers']
    n_vert_all = bg_dict['n_vert_all']
    #bg_all = bg_dict['bg_all']
    avg_bg_ndnf_bin = bg_dict['Ndnf_bin'] 
    avg_bg_sst_bin = bg_dict['Sst_bin']  
    avg_bg_vip_bin = bg_dict['Vip_bin']
    print('Data loaded')
    fdjkl

except:
    
    print('Data not found, calculating background values')
    bg_all = np.ones([n, h, w])
    n_vert_all = {}
    centers = {}

    no_cells = len(Cell.cell_data.keys())
    for cell in range(no_cells):
#     for cell in cells_sub:

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']
        n_vert_all[cell] = {}
        centers[cell] = {}

        for plane in z_planes:

            xy_coord = mask_vertices[cell_no][plane]
            x = xy_coord[0]
            y = xy_coord[1]
            z = plane
            centers[cell][plane] = np.array([np.mean(x), np.mean(y)]).astype(int)


            bg_all[z, x, y] = np.zeros([1, len(x)])

            n_vert_all[cell][plane] = len(x) 
            
    # fig, ax = plt.subplots(nrows = 3, ncols = 20)
    um_per_px = 0.13
    no_cells = len(Cell.cell_data.keys())
    avg_bg_ndnf_bin = np.zeros(no_cells)
    avg_bg_vip_bin = np.zeros(no_cells)
    avg_bg_sst_bin = np.zeros(no_cells)

    local_region = 50 # in um
    local_px = int(local_region/um_per_px/2)

    min_dist = 10 # in um
    min_dist_px = min_dist/um_per_px

    for cell in range(no_cells):
#     for cell in cells_sub:
        cell_no = cell + 1
        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bg_ndnf = np.zeros(len(z_planes))
        bg_vip = np.zeros(len(z_planes))
        bg_sst = np.zeros(len(z_planes))

        p = 0
        for plane in z_planes:

            center = centers[cell][plane]
            n_verts = n_vert_all[cell][plane]

            x1 = np.max([0, center[0] - local_px])
            x2 = np.min([h, center[0] + local_px])
            y1 = np.max([0, center[1] - local_px])
            y2 = np.min([w, center[1] + local_px])

            center_local = [local_px, local_px]

            local_bg = bg_all[plane, x1:x2, y1:y2].astype(bool)
            local_im_ndnf = im_bin_ndnf[plane, x1:x2, y1:y2]
            local_im_sst = im_bin_sst[plane, x1:x2, y1:y2]
            local_im_vip = im_bin_vip[plane, x1:x2, y1:y2]

    #         ax[0, cell].imshow(local_im, vmin = np.min(local_im), vmax = np.max(local_im)/2)

    #         ax[1, cell].imshow(local_bg)

            verts = np.array(np.where(local_bg))
            dist_from_center = np.linalg.norm(verts - np.reshape(center_local, [2, 1]), axis = 0)
            verts = verts[:, dist_from_center > min_dist_px]
            dist_from_center = dist_from_center[dist_from_center > min_dist_px]

            order = np.argsort(dist_from_center)
            n_verts = np.min([n_verts, len(order)])

            x_bg = verts[0][order[0:n_verts]]
            y_bg = verts[1][order[0:n_verts]]

            local_bg = np.zeros(local_bg.shape).astype(bool)
            local_bg[x_bg, y_bg] = np.ones(n_verts)

    #         ax[2, cell].imshow(local_bg)

            bg_ndnf[p] = np.mean(local_im_ndnf[local_bg])
            bg_vip[p] = np.mean(local_im_vip[local_bg])
            bg_sst[p] = np.mean(local_im_sst[local_bg])

            p += 1



        avg_bg_ndnf_bin[cell] = np.mean(bg_ndnf)
        avg_bg_vip_bin[cell] = np.mean(bg_vip)
        avg_bg_sst_bin[cell] = np.mean(bg_sst)
        
    bg_dict['Ndnf_bin'] = avg_bg_ndnf_bin 
    bg_dict['Sst_bin'] = avg_bg_sst_bin 
    bg_dict['Vip_bin'] = avg_bg_vip_bin 

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(bg_dict, f)
            
    #del bg_all

Data loaded
Data not found, calculating background values
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280
Cell 290
Cell 300
Cell 310
Cell 320
Cell 330
Cell 340
Cell 350


### Background in diff images

In [27]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.12\\S5'
save_file = 'S5_background2_ndnf_sst_vip.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        bg_dict = pkl.load(f)   
    
    centers = bg_dict['Centers']
    n_vert_all = bg_dict['n_vert_all']
    bg_all = bg_dict['bg_all']
    avg_bg_ndnf_diff = bg_dict['ndnf_diff'] 
    avg_bg_sst_diff = bg_dict['sst_diff']  
    avg_bg_vip_diff = bg_dict['vip_diff']

    print('Data loaded')

except:
    
    print('Data not found, calculating background values')
    bg_all = np.ones([n, h, w])
    n_vert_all = {}
    centers = {}

    no_cells = len(Cell.cell_data.keys())
    for cell in range(no_cells):

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']
        n_vert_all[cell] = {}
        centers[cell] = {}

        for plane in np.array(z_planes).astype(int):

            xy_coord = mask_vertices[cell_no][plane]
            x = xy_coord[0]
            y = xy_coord[1]
            z = plane
            centers[cell][plane] = np.array([np.mean(x), np.mean(y)]).astype(int)


            bg_all[z, x, y] = np.zeros([1, len(x)])

            n_vert_all[cell][plane] = len(x) 
            
    # fig, ax = plt.subplots(nrows = 3, ncols = 20)
    um_per_px = 0.19
    no_cells = len(Cell.cell_data.keys())
    avg_bg_ndnf_diff = np.zeros(no_cells)
    avg_bg_vip_diff = np.zeros(no_cells)
    avg_bg_sst_diff = np.zeros(no_cells)

    local_region = 50 # in um
    local_px = int(local_region/um_per_px/2)

    min_dist = 10 # in um
    min_dist_px = min_dist/um_per_px

    for cell in range(no_cells):
        cell_no = cell + 1
        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bg_ndnf = np.zeros(len(z_planes))
        bg_vip = np.zeros(len(z_planes))
        bg_sst = np.zeros(len(z_planes))

        p = 0
        for plane in np.array(z_planes).astype(int):

            center = centers[cell][plane]
            n_verts = n_vert_all[cell][plane]

            x1 = np.max([0, center[0] - local_px])
            x2 = np.min([h, center[0] + local_px])
            y1 = np.max([0, center[1] - local_px])
            y2 = np.min([w, center[1] + local_px])

            center_local = [local_px, local_px]

            local_bg = bg_all[plane, x1:x2, y1:y2].astype(bool)
            local_im_ndnf = im_diff_ndnf[plane, x1:x2, y1:y2]
            local_im_sst = im_diff_sst[plane, x1:x2, y1:y2]
            local_im_vip = im_diff_vip[plane, x1:x2, y1:y2]

    #         ax[0, cell].imshow(local_im, vmin = np.min(local_im), vmax = np.max(local_im)/2)

    #         ax[1, cell].imshow(local_bg)

            verts = np.array(np.where(local_bg))
            dist_from_center = np.linalg.norm(verts - np.reshape(center_local, [2, 1]), axis = 0)
            verts = verts[:, dist_from_center > min_dist_px]
            dist_from_center = dist_from_center[dist_from_center > min_dist_px]

            order = np.argsort(dist_from_center)
            n_verts = np.min([n_verts, len(order)])

            x_bg = verts[0][order[0:n_verts]]
            y_bg = verts[1][order[0:n_verts]]

            local_bg = np.zeros(local_bg.shape).astype(bool)
            local_bg[x_bg, y_bg] = np.ones(n_verts)

    #         ax[2, cell].imshow(local_bg)

            bg_ndnf[p] = np.mean(local_im_ndnf[local_bg])
            bg_vip[p] = np.mean(local_im_vip[local_bg])
            bg_sst[p] = np.mean(local_im_sst[local_bg])

            p += 1



        avg_bg_ndnf_diff[cell] = np.mean(bg_ndnf)
        avg_bg_vip_diff[cell] = np.mean(bg_vip)
        avg_bg_sst_diff[cell] = np.mean(bg_sst)
        
    bg_dict['ndnf_bin'] = avg_bg_ndnf_diff 
    bg_dict['sst_bin'] = avg_bg_sst_diff 
    bg_dict['vip_bin'] = avg_bg_vip_diff 

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(bg_dict, f)
            
    del bg_all

Data not found, calculating background values
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280
Cell 290
Cell 300
Cell 310
Cell 320
Cell 330
Cell 340
Cell 350


### Calculate mask centers

In [123]:
bg_all = np.ones([n, h, w])
n_vert_all = {}
centers = {}

no_cells = len(Cell.cell_data.keys())
for cell in range(no_cells):

    cell_no = cell + 1
    cell_dict = Cell.cell_data[cell_no]
    z_planes = cell_dict['z_planes']
    n_vert_all[cell] = {}
    centers[cell] = {}

    for plane in z_planes:

        xy_coord = mask_vertices[cell_no][plane]
        x = xy_coord[0]
        y = xy_coord[1]
        z = plane
        centers[cell][plane] = np.array([np.mean(x), np.mean(y)]).astype(int)


        bg_all[z, x, y] = np.zeros([1, len(x)])

        n_vert_all[cell][plane] = len(x) 

In [78]:
save_file

'S1_background_ndnf_sst_vip.pkl'

In [94]:
with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
            pkl.dump({'Ndnf': avg_bg_ndnf, 'Sst': avg_bg_sst, 'Vip':avg_bg_vip, 'Centers': centers,
                     'n_vert_all': n_vert_all, 'bg_all': bg_all}, f)

In [127]:
del all_cell_pixels

## Use background to estimate significance of signal

### Get bootstrap sample means from background

#### Raw images

##### Check bootstrap samples

In [ ]:
x - 1

In [73]:
signal = int_ndnf - avg_bg_ndnf
order = np.argsort(signal)
npos = pv_ndnf2 < 0.05
indices = order[np.logical_not(npos[order])]
indices = indices[-20:]

cell = indices[0]
bg_bs = bg_bs_ndnf[cell]
planes = bg_bs.keys()
bs_means = []
for plane in planes:
    bs_means = np.concatenate([bs_means, bg_bs[plane]])
    
print('Signal = {0}'.format(int_ndnf[cell]))
print('Background = {0}'.format(avg_bg_ndnf[cell]))
print('Signal - background = {0}'.format(int_ndnf[cell] - avg_bg_ndnf[cell]))
print('Planes: {0}'.format(planes))
print('p value = {0}'.format(pv_ndnf2[cell]))

plt.figure()
plt.hist(bs_means, 100)

Signal = 2998.144271880649
Background = 2096.792077073065
Signal - background = 901.352194807584
Planes: dict_keys([1, 0])
p value = 1.0


(array([  1.,   1.,   0.,   2.,   1.,   3.,   1.,   1.,   1.,   0., 983.,
          6.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   1.,   0., 983.,   8.,   4.,   1.,   1.,   1.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          1.]),
 array([2063.36904418, 2064.200055  , 2065.03106583, 2065.86207665,
        2066.69308747, 2067.5240983 , 2068.35510912, 2069.18611994,
        2070.01713076, 2070.84814159, 2071.67915241, 2072.51016323,
        2073.34117406, 2074.17218488, 2075.0031957 , 2075.83420652,
        2076.66521735, 2077.49622817, 2078.327

##### Get bootstrap values

In [52]:
t0 = time.time()

save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'
save_file = 'S5_bootstrap.pkl'

# Parameters
n_sample = 1000 # Number of samples to calculate bootstrap confidence interval
um_per_px = 0.19
local_region = 50 # in um
min_dist = 10 # in um
bg_factor = 4 # Take 4x the number of pixels in the mask of the cell, for local background - it will still be local but you'll
            # get a better estimate of background by averaging over more pixels.

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        d = pkl.load(f)
    bg_bs_ndnf = d['Ndnf']
    bg_bs_sst = d['Sst']
    bg_bs_vip = d['Vip']
    print('Data loaded')
    
except:
    print('No data found, calculating p values')

    no_cells = len(Cell.cell_data.keys())
    bg_bs_ndnf = {}
    bg_bs_sst = {}
    bg_bs_vip = {}

    local_px = int(local_region/um_per_px/2)
    min_dist_px = min_dist/um_per_px

    for cell in range(no_cells):

        if np.mod(cell, 10) == 0:
            print('Cell {0}: {1} seconds'.format(cell, np.round(time.time() - t0)))

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bg_bs_ndnf[cell] = {}
        bg_bs_sst[cell] = {}
        bg_bs_vip[cell] = {}
        
        p = 0

        for plane in z_planes:

            bg_bs_ndnf[cell][plane] = np.zeros(n_sample)
            bg_bs_sst[cell][plane] = np.zeros(n_sample)
            bg_bs_vip[cell][plane] = np.zeros(n_sample)
            
            center = centers[cell][plane]
            n_verts = n_vert_all[cell][plane]

            x1 = np.max([0, center[0] - local_px])
            x2 = np.min([h, center[0] + local_px])
            y1 = np.max([0, center[1] - local_px])
            y2 = np.min([w, center[1] + local_px])

            center_local = [local_px, local_px]

            local_bg = bg_all[plane, x1:x2, y1:y2].astype(bool)

            verts = np.array(np.where(local_bg))
            dist_from_center = np.linalg.norm(verts - np.reshape(center_local, [2, 1]), 
                                              axis = 0)
            verts = verts[:, dist_from_center > min_dist_px]
            dist_from_center = dist_from_center[dist_from_center > min_dist_px]

            order = np.argsort(dist_from_center)
            n_verts_bg = np.min([bg_factor*n_verts, len(order)])

            verts = verts[:, order[0:n_verts_bg]]
            samples = np.random.choice(np.linspace(0, n_verts_bg - 1, n_verts_bg).astype(int), 
                                       (n_verts, n_sample), replace = True)
            x_bs = verts[0][samples]
            y_bs = verts[1][samples]

            local_im_ndnf = im_array_ndnf[plane, x1:x2, y1:y2]
            local_im_sst = im_array_sst[plane, x1:x2, y1:y2]
            local_im_vip = im_array_vip[plane, x1:x2, y1:y2]


            local_bg_bs = np.zeros([x2 - x1, y2 - y1]).astype(bool)
            for s in range(n_sample):
                local_bg_bs[x_bs[:, s], y_bs[:, s]] = np.ones(n_verts)
                bg_bs_ndnf[cell][plane][s] = np.mean(local_im_ndnf[local_bg_bs])
                bg_bs_vip[cell][plane][s] = np.mean(local_im_vip[local_bg_bs])
                bg_bs_sst[cell][plane][s] = np.mean(local_im_sst[local_bg_bs])

            p += 1

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump({'Ndnf': bg_bs_ndnf, 'Vip': bg_bs_vip, 'Sst': bg_bs_sst}, f)     
    

Data loaded


#### Binarized images

In [62]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'
save_file = 'S5_bootstrap.pkl'

# Parameters
n_sample = 1000 # Number of samples to calculate bootstrap confidence interval
um_per_px = 0.13
local_region = 50 # in um
min_dist = 10 # in um
bg_factor = 10 # Take 10x the number of pixels in the mask of the cell, for local background - it will still be local but you'll
            # get a better estimate of background by averaging over more pixels.

t0 = time.time()
try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        d = pkl.load(f)
    bg_bs_ndnf_bin = d['Ndnf_bin']
    bg_bs_sst_bin = d['Sst_bin']
    bg_bs_vip_bin = d['Vip_bin']
    print('Data loaded')
    jil
    
except:
    print('No data found, calculating p values')

    no_cells = len(Cell.cell_data.keys())
    bg_bs_ndnf_bin = {}
    bg_bs_sst_bin = {}
    bg_bs_vip_bin = {}

    local_px = int(local_region/um_per_px/2)
    min_dist_px = min_dist/um_per_px

    for cell in range(no_cells):
#     for cell in cells_sub:

        if np.mod(cell, 10) == 0:
            t1 = np.round(time.time() - t0)
            print('Cell {0}, {1} seconds'.format(cell, t1))

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        bg_bs_ndnf_bin[cell] = {}
        bg_bs_sst_bin[cell] = {}
        bg_bs_vip_bin[cell] = {}
        
        p = 0

        for plane in z_planes:

            bg_bs_ndnf_bin[cell][plane] = np.zeros(n_sample)
            bg_bs_sst_bin[cell][plane] = np.zeros(n_sample)
            bg_bs_vip_bin[cell][plane] = np.zeros(n_sample)
            
            center = centers[cell][plane]
            n_verts = n_vert_all[cell][plane]

            x1 = np.max([0, center[0] - local_px])
            x2 = np.min([h, center[0] + local_px])
            y1 = np.max([0, center[1] - local_px])
            y2 = np.min([w, center[1] + local_px])

            center_local = [local_px, local_px]

            local_bg = bg_all[plane, x1:x2, y1:y2].astype(bool)

            verts = np.array(np.where(local_bg))
            dist_from_center = np.linalg.norm(verts - np.reshape(center_local, [2, 1]), 
                                              axis = 0)
            verts = verts[:, dist_from_center > min_dist_px]
            dist_from_center = dist_from_center[dist_from_center > min_dist_px]

            order = np.argsort(dist_from_center)
            n_verts_bg = np.min([bg_factor*n_verts, len(order)])

            verts = verts[:, order[0:n_verts_bg]]
            samples = np.random.choice(np.linspace(0, n_verts_bg - 1, n_verts_bg).astype(int), 
                                       (n_verts, n_sample), replace = True)
            x_bs = verts[0][samples]
            y_bs = verts[1][samples]

            local_im_ndnf = im_bin_ndnf[plane, x1:x2, y1:y2]
            local_im_sst = im_bin_sst[plane, x1:x2, y1:y2]
            local_im_vip = im_bin_vip[plane, x1:x2, y1:y2]


            local_bg_bs = np.zeros([x2 - x1, y2 - y1]).astype(bool)
            for s in range(n_sample):
                local_bg_bs[x_bs[:, s], y_bs[:, s]] = np.ones(n_verts)
                bg_bs_ndnf_bin[cell][plane][s] = np.mean(local_im_ndnf[local_bg_bs])
                bg_bs_vip_bin[cell][plane][s] = np.mean(local_im_vip[local_bg_bs])
                bg_bs_sst_bin[cell][plane][s] = np.mean(local_im_sst[local_bg_bs])

            p += 1
            
    d['Ndnf_bin'] = bg_bs_ndnf_bin 
    d['Sst_bin'] = bg_bs_sst_bin 
    d['Vip_bin'] = bg_bs_vip_bin 

    with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(d, f)     
    

Data loaded
No data found, calculating p values
Cell 0, 1.0 seconds
Cell 10, 15.0 seconds
Cell 20, 30.0 seconds
Cell 30, 49.0 seconds
Cell 40, 65.0 seconds
Cell 50, 84.0 seconds
Cell 60, 98.0 seconds
Cell 70, 112.0 seconds
Cell 80, 129.0 seconds
Cell 90, 144.0 seconds
Cell 100, 167.0 seconds
Cell 110, 182.0 seconds
Cell 120, 201.0 seconds
Cell 130, 222.0 seconds
Cell 140, 244.0 seconds
Cell 150, 259.0 seconds
Cell 160, 276.0 seconds
Cell 170, 300.0 seconds
Cell 180, 319.0 seconds
Cell 190, 338.0 seconds
Cell 200, 362.0 seconds
Cell 210, 385.0 seconds
Cell 220, 403.0 seconds
Cell 230, 418.0 seconds
Cell 240, 434.0 seconds
Cell 250, 450.0 seconds
Cell 260, 469.0 seconds
Cell 270, 488.0 seconds
Cell 280, 506.0 seconds
Cell 290, 528.0 seconds
Cell 300, 546.0 seconds
Cell 310, 561.0 seconds
Cell 320, 582.0 seconds
Cell 330, 599.0 seconds
Cell 340, 616.0 seconds
Cell 350, 633.0 seconds


### Calculate p-value  and confidence interval

In [63]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'
save_file1 = 'S5_pvals.pkl'
save_file2 = 'S5_conf_intvls.pkl'
save_file3 = 'S5_pvals_lipo_excl.pkl'
save_file4 = 'S5_pvals_lipo_excl_bin.pkl'

p_value = 0.05 # Threshold for calling signal significantly different from background

try:
    with open('{0}\\{1}'.format(save_loc, save_file1), 'rb') as f:
        p_vals = pkl.load(f)
        pv_ndnf = p_vals['Ndnf']
        pv_sst = p_vals['Sst']
        pv_vip = p_vals['Vip']
    print('Loaded p values for raw images')
    
    with open('{0}\\{1}'.format(save_loc, save_file2), 'rb') as f:
        ci = pkl.load(f)
        ci_ndnf = ci['Ndnf']
        ci_sst = ci['Sst']
        ci_vip = ci['Vip']
    print('Loaded confidence intervals')
    
    with open('{0}\\{1}'.format(save_loc, save_file3), 'rb') as f:
        p_vals = pkl.load(f)
        pv_ndnf2 = p_vals['Ndnf']
        pv_sst2 = p_vals['Sst']
        pv_vip2 = p_vals['Vip']
    print('Loaded p values for lipofuscin excluded masks')
    
    with open('{0}\\{1}'.format(save_loc, save_file4), 'rb') as f:
        p_vals = pkl.load(f)
        pv_ndnf3 = p_vals['Ndnf']
        pv_sst3 = p_vals['Sst']
        pv_vip3 = p_vals['Vip']
    print('Loaded p values for lipofuscin excluded masks in binarized images')
    hu
    
except:
    print('Data not found; calculating')
    
    ci_ndnf = np.zeros(no_cells)
    ci_vip = np.zeros(no_cells)
    ci_sst = np.zeros(no_cells)

    pv_ndnf = np.zeros(no_cells)
    pv_vip = np.zeros(no_cells)
    pv_sst = np.zeros(no_cells)
    
    pv_ndnf2 = np.zeros(no_cells)
    pv_vip2 = np.zeros(no_cells)
    pv_sst2 = np.zeros(no_cells)
    
    pv_ndnf3 = np.ones(no_cells)
    pv_vip3 = np.ones(no_cells)
    pv_sst3 = np.ones(no_cells)
    
    for cell in range(no_cells):
#     for cell in cells_sub:
        
        if np.mod(cell, 10) == 0:
            print('Cell {0}'.format(cell))

        cell_no = cell + 1
        cell_dict = Cell.cell_data[cell_no]
        z_planes = cell_dict['z_planes']

        # Ndnf
        
        bs_means = bg_bs_ndnf[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_ndnf[cell][plane]])
        pv_ndnf[cell] = np.sum(bs_means > avg_intensity_ndnf[cell] - avg_bg_ndnf[cell])/len(bs_means)
        ci_ndnf[cell] = np.mean(np.percentile(bs_means, (1 - p_value)*100, axis = 0))
#         pv_ndnf2[cell] = np.sum(bs_means > int_ndnf[cell] - avg_bg_ndnf[cell])/len(bs_means)
        pv_ndnf2[cell] = np.sum(bs_means >= int_ndnf[cell])/len(bs_means)
        
        bs_means = bg_bs_ndnf_bin[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_ndnf_bin[cell][plane]])
#         pv_ndnf3[cell] = np.sum(bs_means >= int_ndnf_bin[cell] - avg_bg_ndnf_bin[cell])/len(bs_means)
        pv_ndnf3[cell] = np.sum(bs_means >= int_ndnf_bin[cell])/len(bs_means)
        
        # Vip
        
        bs_means = bg_bs_vip[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_vip[cell][plane]])
        pv_vip[cell] = np.sum(bs_means > avg_intensity_vip[cell] - avg_bg_vip[cell])/len(bs_means)
#         pv_vip2[cell] = np.sum(bs_means > int_vip[cell] - avg_bg_vip[cell])/len(bs_means)
        pv_vip2[cell] = np.sum(bs_means >= int_vip[cell])/len(bs_means)
        
        ci_vip[cell] = np.mean(np.percentile(bs_means, (1 - p_value)*100, axis = 0))

        bs_means = bg_bs_vip_bin[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_vip_bin[cell][plane]])
#         pv_vip3[cell] = np.sum(bs_means >= int_vip_bin[cell] - avg_bg_vip_bin[cell])/len(bs_means)
        pv_vip3[cell] = np.sum(bs_means >= int_vip_bin[cell])/len(bs_means)
        
        # Sst
        
        bs_means = bg_bs_sst[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_sst[cell][plane]])
        pv_sst[cell] = np.sum(bs_means > avg_intensity_sst[cell] - avg_bg_sst[cell])/len(bs_means)
#         pv_sst2[cell] = np.sum(bs_means > int_sst[cell] - avg_bg_sst[cell])/len(bs_means)
        pv_sst2[cell] = np.sum(bs_means >= int_sst[cell])/len(bs_means)
        
        ci_sst[cell] = np.mean(np.percentile(bs_means, (1 - p_value)*100, axis = 0))
        
        bs_means = bg_bs_sst_bin[cell][z_planes[0]]
        for plane in z_planes[1:]:
            bs_means = np.concatenate([bs_means, bg_bs_sst_bin[cell][plane]])
#         pv_sst3[cell] = np.sum(bs_means >= int_sst_bin[cell] - avg_bg_sst_bin[cell])/len(bs_means)
        pv_sst3[cell] = np.sum(bs_means >= int_sst_bin[cell])/len(bs_means)


    with open('{0}\\{1}'.format(save_loc, save_file1), 'wb') as f:
        pkl.dump({'Ndnf': pv_ndnf, 'Sst': pv_sst, 'Vip':pv_vip}, f)

    with open('{0}\\{1}'.format(save_loc, save_file2), 'wb') as f:
        pkl.dump({'Ndnf': ci_ndnf, 'Sst': ci_sst, 'Vip':ci_vip}, f)

    with open('{0}\\{1}'.format(save_loc, save_file3), 'wb') as f:
        pkl.dump({'Ndnf': pv_ndnf2, 'Sst': pv_sst2, 'Vip':pv_vip2}, f)

    with open('{0}\\{1}'.format(save_loc, save_file4), 'wb') as f:
        pkl.dump({'Ndnf': pv_ndnf3, 'Sst': pv_sst3, 'Vip':pv_vip3}, f)

    
print('')
print('With raw images:')

print('Ndnf: {0} cells are significant'.format(np.sum(pv_ndnf < 0.05)))
print('Vip: {0} cells are significant'.format(np.sum(pv_vip < 0.05)))
print('Sst: {0} cells are significant'.format(np.sum(pv_sst < 0.05)))

print('')

print('Ndnf & Sst: {0} cells are significant'.format(np.sum(np.logical_and(pv_ndnf < 0.05, pv_sst < 0.05))))
print('Vip & Sst: {0} cells are significant'.format(np.sum(np.logical_and(pv_vip < 0.05, pv_sst < 0.05))))
print('Ndnf & Vip: {0} cells are significant'.format(np.sum(np.logical_and(pv_ndnf < 0.05, pv_vip < 0.05))))

print('')
print('With lipofuscin pixels exluded:')

print('Ndnf: {0} cells are significant'.format(np.sum(pv_ndnf2 < 0.05)))
print('Vip: {0} cells are significant'.format(np.sum(pv_vip2 < 0.05)))
print('Sst: {0} cells are significant'.format(np.sum(pv_sst2 < 0.05)))

print('')

print('Ndnf & Sst: {0} cells are significant'.format(np.sum(np.logical_and(pv_ndnf2 < 0.05, pv_sst2 < 0.05))))
print('Vip & Sst: {0} cells are significant'.format(np.sum(np.logical_and(pv_vip2 < 0.05, pv_sst2 < 0.05))))
print('Ndnf & Vip: {0} cells are significant'.format(np.sum(np.logical_and(pv_ndnf2 < 0.05, pv_vip2 < 0.05))))

print('')
print('With lipofuscin pixels exluded, in binarized images:')

print('Ndnf: {0} cells are significant'.format(np.sum(pv_ndnf3 < 0.05)))
print('Vip: {0} cells are significant'.format(np.sum(pv_vip3 < 0.05)))
print('Sst: {0} cells are significant'.format(np.sum(pv_sst3 < 0.05)))

print('')

print('Ndnf & Sst: {0} cells are significant'.format(np.sum(np.logical_and(pv_ndnf3 < 0.05, pv_sst3 < 0.05))))
print('Vip & Sst: {0} cells are significant'.format(np.sum(np.logical_and(pv_vip3 < 0.05, pv_sst3 < 0.05))))
print('Ndnf & Vip: {0} cells are significant'.format(np.sum(np.logical_and(pv_ndnf3 < 0.05, pv_vip3 < 0.05))))

Loaded p values for raw images
Loaded confidence intervals
Loaded p values for lipofuscin excluded masks
Loaded p values for lipofuscin excluded masks in binarized images
Data not found; calculating
Cell 0
Cell 10
Cell 20
Cell 30
Cell 40
Cell 50
Cell 60
Cell 70
Cell 80
Cell 90
Cell 100
Cell 110
Cell 120
Cell 130
Cell 140
Cell 150
Cell 160
Cell 170
Cell 180
Cell 190
Cell 200
Cell 210
Cell 220
Cell 230
Cell 240
Cell 250
Cell 260
Cell 270
Cell 280
Cell 290
Cell 300
Cell 310
Cell 320
Cell 330
Cell 340
Cell 350

With raw images:
Ndnf: 21 cells are significant
Vip: 32 cells are significant
Sst: 49 cells are significant

Ndnf & Sst: 1 cells are significant
Vip & Sst: 2 cells are significant
Ndnf & Vip: 2 cells are significant

With lipofuscin pixels exluded:
Ndnf: 312 cells are significant
Vip: 323 cells are significant
Sst: 227 cells are significant

Ndnf & Sst: 211 cells are significant
Vip & Sst: 213 cells are significant
Ndnf & Vip: 294 cells are significant

With lipofuscin pixels exlude

## Check classification against manual labels

In [213]:
ndnf_pos2

array([ 18., 115., 107.,   7.,  30., 113., 146.,  54.,   1.,  53.,  21.,
        98.,  57., 124.,   0.,  42.,  13., 118.,  20.])

In [64]:
# Ndnf
#detected = pv_ndnf < 0.05
cells = list(range(Cell.n_cells))
detected = [i for i in cells if pv_ndnf3[i] < 0.05]
manual = ndnf_pos

true_pos = [i for i in detected if i in manual]
false_pos = [i for i in detected if not i in manual]
true_neg = [i for i in cells if np.logical_and(not i in detected, not i in manual)]
false_neg = [i for i in manual if not i in detected]

print('Ndnf')
print('{0} true positives'.format(len(true_pos)))
print('{0} false positives'.format(len(false_pos)))
print('{0} true negative'.format(len(true_neg)))
print('{0} false negative'.format(len(false_neg)))

# Vip
#detected = pv_vip < 0.05
detected = [i for i in cells if pv_vip3[i] < 0.05]
manual = vip_pos

true_pos = [i for i in detected if i in manual]
false_pos = [i for i in detected if not i in manual]
true_neg = [i for i in cells if np.logical_and(not i in detected, not i in manual)]
false_neg = [i for i in manual if not i in detected]

print('Vip')
print('{0} true positives'.format(len(true_pos)))
print('{0} false positives'.format(len(false_pos)))
print('{0} true negative'.format(len(true_neg)))
print('{0} false negative'.format(len(false_neg)))


# Sst
#detected = pv_sst < 0.05
detected = [i for i in cells if pv_sst3[i] < 0.05]
manual = sst_pos

true_pos = [i for i in detected if i in manual]
false_pos = [i for i in detected if not i in manual]
true_neg = [i for i in cells if np.logical_and(not i in detected, not i in manual)]
false_neg = [i for i in manual if not i in detected]

print('Sst')
print('{0} true positives'.format(len(true_pos)))
print('{0} false positives'.format(len(false_pos)))
print('{0} true negative'.format(len(true_neg)))
print('{0} false negative'.format(len(false_neg)))




Ndnf
43 true positives
30 false positives
281 true negative
3 false negative
Vip
33 true positives
27 false positives
295 true negative
2 false negative
Sst
29 true positives
4 false positives
324 true negative
0 false negative


In [100]:
len(cells)

105

## Plot signal and bootstrap for selected cells

In [ ]:
fig, ax = plt.subplots(nrows = 2, ncols = 3)

# Ndnf - positive cells
a = ax[0, 0]
signal = int_ndnf_bin

for cell in ndnf_pos2:
    a.scatter()

# Find cortex surface and calculate depths

In [70]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'
save_file = 'S5_depths.pkl'

try:
    with open('{0}\\{1}'.format(save_loc, save_file), 'rb') as f:
        depths = pkl.load(f)
        print('Depths loaded')
except:
    print('Data not found, draw cortex surface and execute lower cell')

Depths loaded


In [46]:
um_per_px = 0.13
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'
save_file = 'S5_depths.pkl'

def find_depth(d1, d2, cell_center): #d1 and d2 are two points on the surface closest to the cell center
    
    dx = d2[0] - d1[0]
    dy = d2[1] - d1[1]
    d1_cell_x = cell_center[0] - d1[0]
    d1_cell_y = cell_center[1] - d1[1]
    t = (dx * d1_cell_x + dy * d1_cell_y) / (dx * dx + dy * dy)
    px = d1[0] + t * dx # Projection of cell center onto line joining d1 and d2
    py = d1[1] + t * dy
    depth = np.linalg.norm([px - cell_center[0], py - cell_center[1]])
    return depth

# Draw surface and calculate depths
surface = viewer.layers['surface'].data[-1][:, 1:]
depths = np.zeros(Cell.n_cells)

for cell in range(Cell.n_cells):
    pls = centers[cell].keys()
    cx = []
    cy = []
    for pl in pls:
        cx = np.append(cx, centers[cell][pl][0])
        cy = np.append(cy, centers[cell][pl][1])
    c = [np.mean(cx), np.mean(cy)]
    distances = np.linalg.norm(c - surface, axis = 1)
    order = np.argsort(distances)
    [p1, p2] = order[0:2]
    d1 = surface[p1]
    d2 = surface[p2]
    depths[cell] = find_depth(d1, d2, c)
depths = depths*um_per_px

with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(depths, f)


In [55]:
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5'
save_file = 'S5_depths.pkl'

with open('{0}\\{1}'.format(save_loc, save_file), 'wb') as f:
        pkl.dump(depths, f)

# Plots

## 2D scatter plot of pixel values (pairs of channels)

In [65]:
img_dict = {'Ndnf': im_bin_ndnf_lipo, 'Vip': im_bin_vip_lipo}
# img_dict = {'Ndnf': im_bin_ndnf_lipo, 'Sst': im_bin_sst_lipo, 'Vip': im_bin_vip_lipo}
#img_dict = {'Ndnf': im_diff_ndnf, 'Sst': im_diff_sst, 'Vip': im_diff_vip}
#img_dict = {'Ndnf': im_array_ndnf, 'Sst': im_array_sst, 'Vip': im_array_vip}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}
#pixels_dict = {'Cells': all_cell_pixels}

colors_dict = {'Cells': 'b', 'Lipofuscin': 'k'}
title = 'Diff gauss sigma = ({0}, {1}); binarized above {2} std from median'.format(sigma_small, sigma_large, thresh_scale)
#title = 'Raw images'
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5\\Plots'
save_file = '{0} scatter lipo + cells.png'.format(title)
#save_file = '{0} scatter cells.png'.format(title)

scatter_channels.scatter_channels(img_dict, pixels_dict, max_per_group= 100000, colors_dict = colors_dict, 
                                  title = title, save = True, save_loc = save_loc, save_file = save_file)

del img_dict
del pixels_dict

## 3D scatter plot of pixel values (3 channels)

In [253]:
len(m_dist_vals['Cells'] > thresh)

724480

In [54]:
len(all_lipo_pixels)

854

In [35]:
t0 = time.time()
n = len(all_cell_pixels)
cell_px_is_lipo = np.zeros(n)
for px in range(n):
    pc = all_cell_pixels[px]
    if im_array_lipo[pc[0], pc[1], pc[2]] == 1:
        cell_px_is_lipo[px] = 1

In [38]:
#img_dict = {'Ndnf': im_bin_ndnf, 'Gad1': im_bin_gad1, 'Vip': im_bin_vip}
img_dict = {'546': im_bin_ndnf_lipo, '647': im_bin_sst_lipo, '594': im_bin_vip_lipo}
#img_dict = {'Ndnf': im_diff_ndnf_lipo, 'Sst': im_diff_sst_lipo, 'Vip': im_diff_vip_lipo}
#img_dict = {'Ndnf': im_bin_ndnf, 'Sst': im_bin_sst, 'Vip': im_bin_vip}
#img_dict = {'546': im_array_ndnf, '647': im_array_sst, '594': im_array_vip}
#img_dict = {'546': im_ndnf_lipo, '647': im_sst_lipo, '594': im_vip_lipo}


#pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}
pixels_dict = {'Cells': all_cell_pixels}
#pixels_dict = {'Cells': lipo_pixels, 'Lipofuscin': all_lipo_pixels}
#pixels_dict = {'Lipofuscin': all_lipo_pixels}


#colors_dict = {'Cells': 'k', 'Lipofuscin': 'b'}
#colors_dict = {'Cells': m_dist_vals['Cells'], 'Lipofuscin': m_dist_vals['Lipofuscin']}
#colors_dict = {'Cells': np.log(m_dist_vals['Cells']), 'Lipofuscin': 'r'}
#colors_dict ={'Cells': np.logical_not(m_dist_vals['Cells'] > thresh), 'Lipofuscin': 'r'}
#colors_dict = {'Cells': np.logical_not(np.log(mean_vals) < 5), 'Lipofuscin': 'r'}
#colors_dict = {'Cells': np.logical_not([a in lipo_pixels for a in all_cell_pixels])}
colors_dict = {'Cells': cell_px_is_lipo}

alpha_dict = {'Cells': 0.4, 'Lipofuscin': 1}

#title = 'Sub-sampled diff gauss sigma = ({0}, {1}) \n binarized above {2} std from median \n color = log mahalanobis distance from lipofuscin pixels'.format(sigma_small, sigma_large, thresh_scale)
#title = 'Raw images'

save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR 3.11\\S5\\Plots'
#save_file = '3D scatter lipo + cells mahalanobis distance2.png'
#save_file = '{0} 3D scatter cells + lipo.png'.format(title)
#save_file = '{0} 3D scatter lipo.png'.format(title)
save_file=  '3D scatter lipo + cells.png'

scatter_channels2(img_dict, pixels_dict, max_per_group= 100000, colors_dict = colors_dict,
                                  alpha_dict = alpha_dict,
                                  make_3D = True,
#                                   title = title, 
#                                   save = True, 
                                  save_loc = save_loc, save_file = save_file)

del img_dict
del pixels_dict

In [47]:
save_file

'3D scatter lipo + cells mahalanobis distance2.png'

In [41]:
plt.close('all')

#### Function definition

In [37]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 27 15:36:38 2020

Input:
    img_dict: Dictionary in which each key is the name of a channel and
              the corresponding value is the image of shape [n_planes, height, width] for that channel
    pixels_dict: Dictionary in which each key is the name of a group of pixels and
                 the corresponding value is an array of pixel coordinates of shape [n_pixels, 3]
    colors_dict: (optional) Dictionary in which each key is the name of a group of pixels and
                 the corresponding value is a color
    max_per_group: Default 10000. Maximum number of pixels per group to plot.
                   If number of pixels exceeds this, pixels will be randomly sub-sampled.


@author: Amrita Singh
"""

from itertools import combinations
import matplotlib.pyplot as plt
import numpy as np
from os.path import sep


def scatter_channels2(img_dict, pixels_dict, max_per_group = 10000, colors_dict = None,
                     make_3D = False, alpha_dict = None,
                     title = None, save = False, save_loc = None, save_file = None, log_scale = False):


    # %%
    channel_names = list(img_dict.keys())
    n_channels = len(channel_names)
    channels = range(n_channels)

    if make_3D:
        combos = list(combinations(channels, 3))
    else:
        combos = list(combinations(channels, 2))

    n_combos = len(combos)

    px_groups = list(pixels_dict.keys())


    # %% Get values for each channel for each pixel group
    px_vals = {}

    for group in px_groups:

        px = pixels_dict[group]
        n_px = px.shape[0]

        # Sub-sample if too many pixels to plot
        if n_px > max_per_group:
            idx = np.random.randint(int(n_px/max_per_group), size = n_px)
            px = px[idx == 0, :]
            if type(colors_dict[group]) is np.ndarray:
                if colors_dict[group].shape[0] > max_per_group:
                    colors_dict[group] = colors_dict[group][idx == 0]

        px_vals[group] = {}

        for channel in channel_names:
            px_vals[group][channel] = img_dict[channel][(px[:, 0], px[:, 1], px[:, 2])]

    # %%
    fig, ax = plt.subplots(nrows = n_combos if n_combos > 1 else 2, ncols = 1, figsize = (20, 20))
    for combo in range(n_combos):

        if make_3D:
            c3 = channel_names[combos[combo][2]]
            ax[combo] = plt.axes(projection='3d')
            ax[combo].set_zlabel(c3, fontsize = 18)

        c1 = channel_names[combos[combo][0]]
        c2 = channel_names[combos[combo][1]]

        ax[combo].set_xlabel(c1, fontsize = 18)
        ax[combo].set_ylabel(c2, fontsize = 18)



        for group in px_groups:

            if log_scale:
                ax[combo].set_xscale('log')
                ax[combo].set_yscale('log')
                if make_3D:
                    ax[combo].set_zscale('log')

            if colors_dict == None:
                if make_3D:
                    p = ax[combo].scatter3D(px_vals[group][c1], px_vals[group][c2], px_vals[group][c3], label = group,
                      marker = '.', alpha = alpha_dict[group])
                    fig.colorbar(p)
                else:
                    ax[combo].scatter(px_vals[group][c1], px_vals[group][c2], label = group, marker = '.', alpha = alpha_dict[group])
            else:
                if make_3D:
                    ax[combo].scatter3D(px_vals[group][c1], px_vals[group][c2], px_vals[group][c3], label = group,
                      marker = '.', alpha = alpha_dict[group], c = colors_dict[group],
                                       cmap = 'bwr'
                                       )
                else:
                    ax[combo].scatter(px_vals[group][c1], px_vals[group][c2], label = group, c = colors_dict[group], marker = '.', alpha = alpha_dict[group])


        ax[combo].legend(fontsize = 18)


    if not title == None:
        ax[0].set_title(title, fontsize = 18)

    if save:
        if save_loc == None:
            print('Please provide save location')
        else:
            if save_file == None:
                print('Please provide save file name')
            else:
                plt.savefig('{0}{1}{2}'.format(save_loc, sep, save_file))


### Get colorbar (has to be in a different figure)

In [24]:
plt.figure()
a = np.zeros([4, 4])
a[0, 0] = np.max(np.log(m_dist_vals['Cells']))
a[0, 1] = np.min(np.log(m_dist_vals['Cells']))
plt.imshow(a)
plt.colorbar(label = 'Log of mahalanobis distance from lipofuscin pixels')

## Histogram of pixel values in each channel

In [25]:
#img_dict = {'Ndnf': im_diff_ndnf, 'Sst': im_diff_sst, 'Vip': im_diff_vip}
img_dict = {'Ndnf': im_array_ndnf, 'Sst': im_array_sst, 'Vip': im_array_vip}

pixels_dict = {'Cells': all_cell_pixels, 'Lipofuscin': all_lipo_pixels}
#pixels_dict = {'Cells': all_cell_pixels}
colors_dict = {'Cells': 'b', 'Lipofuscin': 'k'}
#title = 'Diff gauss sigma = ({0}, {1})'.format(sigma_small, sigma_large)
title = 'Raw images'
save_loc = 'G:\\Shared drives\\as_share\\HCR\\HCR_10.17\\S1_scatter_plots'
save_file = '{0} hist lipo + cells.png'.format(title)
#save_file = '{0} hist cells.png'.format(title)

hist_channels.hist_channels(img_dict, pixels_dict, max_per_group= 100000, colors_dict = colors_dict, 
                            title = title, save = True, save_loc = save_loc, save_file = save_file)

del img_dict
del pixels_dict

## Signal vs depth

### Using p-value for significance

In [78]:
# Plot intensity vs depth for all channels
fig, ax = plt.subplots(nrows = 3, ncols = 3)
    
d = depths

# npos = pv_ndnf < 0.05
# vpos = pv_vip < 0.05
# spos = pv_sst < 0.05

# n = avg_intensity_ndnf
# v = avg_intensity_vip
# s = avg_intensity_sst

# bn = avg_bg_ndnf
# bv = avg_bg_vip
# bs = avg_bg_sst

# ax[0, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
# ax[0, 0].scatter(d[npos], n[npos] - bn[npos], color = 'b', marker = '.')
# #ax[0, 0].set_xlabel('Depth from pia (um)')
# ax[0, 0].set_ylabel('Intensity in channel (bold => p < 0.05)')
# ax[0, 0].set_title('Ndnf: all pixels')

# ax[0, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
# ax[0, 1].scatter(d[vpos], v[vpos] - bv[vpos], color = 'r', marker = '.')
# #ax[0, 1].set_xlabel('Depth from pia (um)')
# ax[0, 1].set_title('Vip: all pixels')

# ax[0, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
# ax[0, 2].scatter(d[spos], s[spos] - bs[spos], color = 'g', marker = '.')
# #ax[0, 2].set_xlabel('Depth from pia (um)')
# ax[0, 2].set_title('Sst: all pixels')

# n = int_ndnf
# v = int_vip
# s = int_sst

# npos2 = pv_ndnf2 < 0.05
# vpos2 = pv_vip2 < 0.05
# spos2 = pv_sst2 < 0.05

# ax[1, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
# ax[1, 0].scatter(d[npos2], n[npos2] - bn[npos2], color = 'b', marker = '.')
# #ax[1, 0].set_xlabel('Depth from pia (um)')
# ax[1, 0].set_ylabel('Intensity in channel (bold => p < 0.05)')
# ax[1, 0].set_title('Ndnf: lipofuscin pixels excluded')

# ax[1, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
# ax[1, 1].scatter(d[vpos2], v[vpos2] - bv[vpos2], color = 'r', marker = '.')
# #ax[1, 1].set_xlabel('Depth from pia (um)')
# ax[1, 1].set_title('Vip: lipofuscin pixels excluded')

# ax[1, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
# ax[1, 2].scatter(d[spos2], s[spos2] - bs[spos2], color = 'g', marker = '.')
# #ax[1, 2].set_xlabel('Depth from pia (um)')
# ax[1, 2].set_title('Sst: lipofuscin pixels excluded')

n = int_ndnf_bin
v = int_vip_bin
s = int_sst_bin

bn = avg_bg_ndnf_bin
bv = avg_bg_vip_bin
bs = avg_bg_sst_bin

npos3 = pv_ndnf3 < 0.05
vpos3 = pv_vip3 < 0.05
spos3 = pv_sst3 < 0.05

# vpos3 = int_vip_bin - avg_bg_vip_bin > 200
# spos3 = int_sst_bin - avg_bg_sst_bin > 500


ax[2, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
ax[2, 0].scatter(d[npos3], n[npos3] - bn[npos3], color = 'b', marker = '.')
ax[2, 0].set_xlabel('Depth from pia (um)')
ax[2, 0].set_ylabel('Intensity in channel (bold => p < 0.05)')
ax[2, 0].set_title('Ndnf: lipofuscin pixels excluded; Binarized images')

ax[2, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
ax[2, 1].scatter(d[vpos3], v[vpos3] - bv[vpos3], color = 'r', marker = '.')
ax[2, 1].set_xlabel('Depth from pia (um)')
ax[2, 1].set_title('Vip: lipofuscin pixels excluded; Binarized images')

ax[2, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
ax[2, 2].scatter(d[spos3], s[spos3] - bs[spos3], color = 'g', marker = '.')
ax[2, 2].set_xlabel('Depth from pia (um)')
ax[2, 2].set_title('Sst: lipofuscin pixels excluded; Binarized images')


Text(0.5, 1.0, 'Sst: lipofuscin pixels excluded; Binarized images')

### Using 95% confidence interval for significance

In [100]:
# Plot intensity vs depth for all channels
fig, ax = plt.subplots(nrows = 2, ncols = 3)
    
d = depths

n = avg_intensity_ndnf
v = avg_intensity_vip
s = avg_intensity_sst

bn = avg_bg_ndnf
bv = avg_bg_vip
bs = avg_bg_sst

npos = n - bn > ci_ndnf
vpos = v - bv > ci_vip
spos = s - bs > ci_sst

ax[0, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
ax[0, 0].scatter(d[npos], n[npos] - bn[npos], color = 'b', marker = '.')
ax[0, 0].set_xlabel('Depth from pia (um)')
ax[0, 0].set_ylabel('Intensity in channel (bold => 95% CI)')
ax[0, 0].set_title('Ndnf: all pixels')

ax[0, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
ax[0, 1].scatter(d[vpos], v[vpos] - bv[vpos], color = 'r', marker = '.')
ax[0, 1].set_xlabel('Depth from pia (um)')
ax[0, 1].set_title('Vip: all pixels')

ax[0, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
ax[0, 2].scatter(d[spos], s[spos] - bs[spos], color = 'g', marker = '.')
ax[0, 2].set_xlabel('Depth from pia (um)')
ax[0, 2].set_title('Sst: all pixels')

n = int_ndnf
v = int_vip
s = int_sst

npos = n - bn > ci_ndnf
vpos = v - bv > ci_vip
spos = s - bs > ci_sst

ax[1, 0].scatter(d, n - bn, color = 'b', alpha = 0.1, marker = '.')
ax[1, 0].scatter(d[npos2], n[npos2] - bn[npos2], color = 'b', marker = '.')
ax[1, 0].set_xlabel('Depth from pia (um)')
ax[1, 0].set_ylabel('Intensity in channel (bold => 95% CI)')
ax[1, 0].set_title('Ndnf: lipofuscin pixels excluded')

ax[1, 1].scatter(d, v - bv, color = 'r', alpha = 0.1, marker = '.')
ax[1, 1].scatter(d[vpos2], v[vpos2] - bv[vpos2], color = 'r', marker = '.')
ax[1, 1].set_xlabel('Depth from pia (um)')
ax[1, 1].set_title('Vip: lipofuscin pixels excluded')

ax[1, 2].scatter(d, s - bs, color = 'g', alpha = 0.1, marker = '.')
ax[1, 2].scatter(d[spos2], s[spos2] - bs[spos2], color = 'g', marker = '.')
ax[1, 2].set_xlabel('Depth from pia (um)')
ax[1, 2].set_title('Sst: lipofuscin pixels excluded')


Text(0.5, 1.0, 'Sst: lipofuscin pixels excluded')

## Raw images of all cells

### Function for plotting

In [102]:
def view_all_cells(n_cells, n_rows, n_cols, signal, im_array, cell_data, size_um, um_per_px, h, w, 
                  save_loc, title, max_intensity = None, spine_color_pos = 'r', pos_cells = [], spine_width = 2,
                  show_masks = False):

    cell_ids = np.array(list(cell_data.keys()))
    cell_order = cell_ids[np.flip(np.argsort(signal)).astype(int)]
    
    size_px = int(size_um/um_per_px/2)
    mask_dil =  np.zeros([h, w]).astype(bool)
    
    if max_intensity == None:
        max_intensity = np.max(signal)
    min_intensity = np.min(im_array)

    t0 = time.time()
    fig, ax = plt.subplots(nrows = n_rows, ncols = n_cols, figsize = (30, 20))

    for cell in range(n_cells):   

        if np.mod(cell, 10) == 0:
            print('Cell {0}: {1} seconds'.format(cell, np.round(time.time() - t0)))

        row = int(cell/n_cols)
        col = np.mod(cell, n_cols)

        cell_id = cell_order[cell]

        planes = cell_data[cell_id]['z_planes']
        med_plane = int(np.median(planes))

        mask = cell_data[cell_id]['masks'][med_plane]
        [x_center, y_center] = np.mean(mask, 0).astype(int)
        [x1, y1] = [np.max([0, x_center - size_px]), np.max([0, y_center - size_px])]
        [x2, y2] = [np.min([h, x_center + size_px]), np.min([w, y_center + size_px])]
        mask_dil[x1:x2, y1:y2] = np.ones([x2 - x1, y2 - y1])

        small_im = np.reshape(im_array[med_plane, mask_dil], [np.min([x2 - x1, size_px*2]), np.min([y2 - y1, size_px*2])])
        ax[row, col].imshow(small_im, vmin = min_intensity, vmax = max_intensity)

        mask_dil[x1:x2, y1:y2] = np.zeros([x2 - x1, y2 - y1])
        
        if show_masks:
            
            cell_px = cell_pixels_no_lipo[cell_id][med_plane]
            cell_mask = np.zeros(small_im.shape)
            x = cell_px[0, :] - x1
            y = cell_px[1, :] - y1
#             for i in range(len(x)):
#                 cell_mask[x[i], y[i]] = 1
            cell_mask[x, y] = np.ones(len(x))
            ax[row, col].imshow(cell_mask, vmin = 0, vmax = 1, cmap = 'gray', alpha = 0.2)
            

#         ax[row, col].axis('off')
        if col == 0:
            ax[row, col].set_ylabel('{0}'.format(np.round(signal[cell], 2)))
#             ax[row, col].axis('on')
        
        if len(pos_cells) > 0:
            if pos_cells[cell] == 1:
                ax[row, col].axis('on')
                ax[row, col].spines['bottom'].set_color(spine_color_pos)
                ax[row, col].spines['top'].set_color(spine_color_pos)
                ax[row, col].spines['left'].set_color(spine_color_pos)
                ax[row, col].spines['right'].set_color(spine_color_pos)
                ax[row, col].spines['bottom'].set_linewidth(spine_width)
                ax[row, col].spines['top'].set_linewidth(spine_width)
                ax[row, col].spines['left'].set_linewidth(spine_width)
                ax[row, col].spines['right'].set_linewidth(spine_width)
        
        ax[row, col].tick_params(axis = 'both', which = 'both', bottom = False, top = False, left = False, 
                                 right = False, labeltop = False, labelright = False, labelleft = False,
                                labelbottom = False)
                
                
#         plt.tick_params(
#     axis='x',          # changes apply to the x-axis
#     which='both',      # both major and minor ticks are affected
#     bottom=False,      # ticks along the bottom edge are off
#     top=False,         # ticks along the top edge are off
#     labelbottom=False) # labels along the bottom edge are off

    
    plt.savefig('{0}{1}{2}.png'.format(save_loc, sep, title))


### Get centers and med planes

In [103]:
# All cells
centers_med_planes = np.zeros([Cell.n_cells, 2])
med_planes = np.zeros(Cell.n_cells)

for cell in range(Cell.n_cells):
    if np.mod(cell, 10) == 0:
        print(cell)
    cell_id = cell + 1
    planes = Cell.cell_data[cell_id]['z_planes']
    med_planes[cell] = int(np.median(planes))
    
    mask = Cell.cell_data[cell_id]['masks'][med_planes[cell]]
    centers_med_planes[cell, :] = np.mean(mask, 0).astype(int)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350


In [121]:
# Selected cells
centers_med_planes = np.zeros([len(cells), 2])
med_planes = np.zeros(len(cells))

for cell in range(len(cells)):
    if np.mod(cell, 10) == 0:
        print(cell)
    cell_id = cells[cell] + 1
    planes = Cell.cell_data[cell_id]['z_planes']
    med_planes[cell] = int(np.median(planes))
    
    mask = Cell.cell_data[cell_id]['masks'][med_planes[cell]]
    centers_med_planes[cell, :] = np.mean(mask, 0).astype(int)

0
10
20
30
40
50
60
70
80
90
100


### Run for different conditions

In [108]:
save_loc = 'G:{0}Shared drives{0}as_share{0}HCR{0}HCR 3.11{0}S5{0}Plots'.format(sep)

titles = {
            1: 'Ndnf_raw_images_all_cells2',
#           2: 'Ndnf_diff_images_all_cells',
#           3: 'Ndnf_bin_images_all_cells2',
          4: 'Sst_raw_images_all_cells2',
#           5: 'Sst_diff_images_all_cells',
#           6: 'Sst_bin_images_all_cells2',
          7: 'Vip_raw_images_all_cells2',
#           8: 'Vip_diff_images_all_cells',
#           9: 'Vip_bin_images_all_cells2',
         }

im_arrays = {
             1: im_array_ndnf,
#              2: im_diff_ndnf,
#              3: im_bin_ndnf,
             4: im_array_sst,
#              5: im_diff_sst,
#              6: im_bin_sst,
             7: im_array_vip,
#              8: im_diff_vip,
#              9: im_bin_vip,
            }
             
signals = {
             1: int_ndnf_bin - avg_bg_ndnf_bin,
#            1: avg_intensity_ndnf - avg_bg_ndnf,
#            2: int_ndnf_diff - avg_bg_ndnf_diff,
#            3: int_ndnf_bin[cells] - avg_bg_ndnf_bin[cells],
           4: int_sst_bin - avg_bg_sst_bin,
#             4: avg_intensity_sst - avg_bg_sst,
#            5: int_sst_diff - avg_bg_sst_diff,
#            6: int_sst_bin[cells] - avg_bg_sst_bin[cells],
            7: int_vip_bin - avg_bg_vip_bin,
#            7: avg_intensity_vip - avg_bg_vip,
#            8: int_vip_diff - avg_bg_vip_diff,
#            9: int_vip_bin[cells] - avg_bg_vip_bin[cells],
          }

pos = {
        1: pv_ndnf3 < 0.05,
        3: pv_ndnf3 < 0.05,
        4: pv_sst3 < 0.05,
        6: pv_sst3 < 0.05,
        7: pv_vip3 < 0.05,
        9: pv_vip3 < 0.05
    
}

n_rows = 12
n_cols = 30

size_um = 30 # In um
um_per_px = 0.13 # Size of a pixel in um

conds = [1, 4, 7]
#conds = [3, 6, 9]
order = {}

#c_data = {cells[i] + 1: Cell.cell_data[cells[i] + 1] for i in range(len(cells))}
c_data = Cell.cell_data

for cond in conds:
    print(titles[cond])
    view_all_cells(Cell.n_cells, n_rows, n_cols, signals[cond], im_arrays[cond],
                   c_data, size_um, um_per_px,
                    h, w, save_loc, titles[cond], 
                   max_intensity = np.max(avg_intensity_ndnf[cells] - avg_bg_ndnf_bin[cells]),
                  pos_cells = pos[cond],
                  spine_color_pos = 'r',
                  spine_width = 5, show_masks = True)
    
    




Ndnf_raw_images_all_cells2
Cell 0: 4.0 seconds
Cell 10: 5.0 seconds
Cell 20: 6.0 seconds
Cell 30: 7.0 seconds
Cell 40: 8.0 seconds
Cell 50: 9.0 seconds
Cell 60: 10.0 seconds
Cell 70: 11.0 seconds
Cell 80: 12.0 seconds
Cell 90: 13.0 seconds
Cell 100: 14.0 seconds
Cell 110: 15.0 seconds
Cell 120: 16.0 seconds
Cell 130: 17.0 seconds
Cell 140: 18.0 seconds
Cell 150: 19.0 seconds
Cell 160: 20.0 seconds
Cell 170: 21.0 seconds
Cell 180: 22.0 seconds
Cell 190: 23.0 seconds
Cell 200: 24.0 seconds
Cell 210: 25.0 seconds
Cell 220: 26.0 seconds
Cell 230: 27.0 seconds
Cell 240: 28.0 seconds
Cell 250: 29.0 seconds
Cell 260: 30.0 seconds
Cell 270: 31.0 seconds
Cell 280: 32.0 seconds
Cell 290: 33.0 seconds
Cell 300: 34.0 seconds
Cell 310: 35.0 seconds
Cell 320: 36.0 seconds
Cell 330: 37.0 seconds
Cell 340: 38.0 seconds
Cell 350: 39.0 seconds
Sst_raw_images_all_cells2
Cell 0: 4.0 seconds
Cell 10: 5.0 seconds
Cell 20: 6.0 seconds
Cell 30: 7.0 seconds
Cell 40: 8.0 seconds
Cell 50: 9.0 seconds
Cell 60: 10

In [212]:
plt.close('all')

### Select positive and negative cells for each gene

In [142]:
ind = np.flip(np.argsort(signals[7]))
cell = ind[46]
print(cell)

315


In [57]:
save_loc = 'G:{0}Shared drives{0}as_share{0}HCR{0}HCR 3.11{0}S5'.format(sep)
save_file = 'Ndnf_sst_vip_pos_cells.pkl'

try:
    with open('{0}{1}{2}'.format(save_loc, sep, save_file), 'rb') as f:
        pos = pkl.load(f)
        ndnf_pos = pos['Ndnf_pos']
        sst_pos = pos['Sst_pos']
        vip_pos = pos['Vip_pos']
    print('Data loaded')

except:
    # Ndnf
    ind = np.flip(np.argsort(signals[1]))
    ndnf_pos = ind[0:40]
    ndnf_pos = np.concatenate([ndnf_pos, [ind[i] for i in [42, 43, 44, 45, 50, 59]]])

    # Sst
    ind = np.flip(np.argsort(signals[4]))
    sst_pos = np.array(ind[0:29])

    # Vip
    ind = np.flip(np.argsort(signals[7]))
    vip_pos = np.array(ind[0:35])

    with open('{0}{1}{2}'.format(save_loc, sep, save_file), 'wb') as f:
        pkl.dump({'Ndnf_pos': ndnf_pos, 'Sst_pos': sst_pos, 'Vip_pos': vip_pos}, f)


Data loaded


##### Check exclusivity

In [19]:
ndnf_vip = [i for i in ndnf_pos if i in vip_pos]
ndnf_sst = [i for i in ndnf_pos if i in sst_pos]
vip_sst = [i for i in vip_pos if i in sst_pos]
print(ndnf_vip)
print(ndnf_sst)
print(vip_sst)

[]
[]
[162]


#### Get planes of positive cells

In [74]:
planes = []
ndnf_pos2 = []
vip_pos2 = []
sst_pos2 = []
for cell in ndnf_pos:
    cell_no = cell + 1
    cp = Cell.cell_data[cell_no]['z_planes']
    planes = np.append(planes, cp)
    if all(x in [0, 1] for x in cp):
        ndnf_pos2 = np.append(ndnf_pos2, cell)
    
for cell in vip_pos:
    cell_no = cell + 1
    cp = Cell.cell_data[cell_no]['z_planes']
    planes = np.append(planes, cp)
    if all(x in [0, 1] for x in cp):
        vip_pos2 = np.append(vip_pos2, cell)
        
for cell in sst_pos:
    cell_no = cell + 1
    cp = Cell.cell_data[cell_no]['z_planes']
    planes = np.append(planes, cp)
    if all(x in [0, 1] for x in cp):
        sst_pos2 = np.append(sst_pos2, cell)
        
plt.figure()
plt.hist(planes)

(array([30., 29., 26., 28.,  0., 23., 26., 18.,  7.,  1.]),
 array([0. , 0.8, 1.6, 2.4, 3.2, 4. , 4.8, 5.6, 6.4, 7.2, 8. ]),
 <a list of 10 Patch objects>)

In [39]:
all(x in [0, 1] for x in cp)

False

In [41]:
ndnf_pos2

array([ 18.,   7.,  30., 146.,  54.,   1.,  53.,  21.,  57.,   0.,  42.,
        13.,  20.])

In [42]:
vip_pos2

array([ 16.,  69.,  12., 110.,  55., 106.,  11., 102.,  34.,  65.,  25.,
        33.])

In [43]:
sst_pos2

array([ 37.,  63.,  44.,  22., 120.,  84.,  39., 125., 132.])

# View depths and signal levels of specific cells

## Ndnf+ cells

In [73]:
sig = int_ndnf_bin - avg_bg_ndnf_bin
max_sig = 10
all_cells = np.array(list(Cell.cell_data.keys())) - 1
cells = all_cells[np.logical_and(sig < max_sig, pv_ndnf3 < 0.05)]

for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv_ndnf3[cell], decimals = 2)))
    

Cell 0: depth 20.26um, signal = 2.02, p value = 0.0
Cell 28: depth 279.1um, signal = 4.78, p value = 0.0
Cell 31: depth 53.79um, signal = 6.24, p value = 0.0
Cell 42: depth 289.02um, signal = 4.6, p value = 0.0
Cell 63: depth 229.51um, signal = 4.8, p value = 0.0
Cell 70: depth 382.53um, signal = 4.88, p value = 0.0
Cell 72: depth 74.01um, signal = 0.77, p value = 0.0
Cell 84: depth 351.77um, signal = 1.05, p value = 0.0
Cell 116: depth 63.59um, signal = 0.49, p value = 0.0
Cell 125: depth 366.83um, signal = 1.38, p value = 0.0
Cell 167: depth 160.16um, signal = 0.12, p value = 0.0
Cell 177: depth 143.9um, signal = -0.28, p value = 0.0
Cell 194: depth 371.96um, signal = 1.97, p value = 0.0
Cell 195: depth 29.29um, signal = 4.78, p value = 0.0
Cell 197: depth 42.24um, signal = 1.78, p value = 0.0
Cell 199: depth 115.88um, signal = 1.44, p value = 0.0
Cell 206: depth 291.01um, signal = 0.25, p value = 0.0
Cell 210: depth 92.45um, signal = 2.08, p value = 0.0
Cell 211: depth 47.88um, sign

## Ndnf- cells

In [75]:
sig = int_ndnf_bin - avg_bg_ndnf_bin
max_sig = 0
all_cells = np.array(list(Cell.cell_data.keys())) - 1
cells = all_cells[np.logical_and(sig < max_sig, pv_ndnf3 > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv_ndnf3[cell], decimals = 2)))


Cell 305: depth 45.01um, signal = -0.21, p value = 1.0
Cell 290: depth 123.08um, signal = -1.23, p value = 1.0
Cell 109: depth 177.52um, signal = -2.34, p value = 1.0
Cell 139: depth 302.68um, signal = -0.15, p value = 1.0
Cell 295: depth 233.11um, signal = -1.17, p value = 1.0
Cell 278: depth 200.48um, signal = -0.2, p value = 1.0
Cell 248: depth 392.49um, signal = -0.24, p value = 1.0
Cell 286: depth 314.93um, signal = -0.84, p value = 1.0
Cell 180: depth 204.11um, signal = -1.32, p value = 1.0
Cell 291: depth 220.63um, signal = -0.57, p value = 1.0


In [76]:
sig = int_ndnf_bin - avg_bg_ndnf_bin
max_sig = 10
all_cells = np.array(list(Cell.cell_data.keys())) - 1
cells = all_cells[np.logical_and(sig < max_sig, pv_ndnf3 > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv_ndnf3[cell], decimals = 2)))


Cell 109: depth 177.52um, signal = -2.34, p value = 1.0
Cell 288: depth 346.37um, signal = 0.0, p value = 1.0
Cell 126: depth 112.38um, signal = 0.0, p value = 1.0
Cell 161: depth 80.02um, signal = 0.0, p value = 1.0
Cell 43: depth 292.14um, signal = 0.0, p value = 1.0
Cell 281: depth 144.53um, signal = 0.0, p value = 1.0
Cell 79: depth 275.98um, signal = 0.0, p value = 1.0
Cell 122: depth 166.83um, signal = 0.0, p value = 1.0
Cell 299: depth 257.23um, signal = -0.35, p value = 1.0
Cell 55: depth 76.89um, signal = 0.0, p value = 1.0


In [90]:
sig = int_ndnf_bin - avg_bg_ndnf_bin
min_sig = 0
all_cells = np.array(list(Cell.cell_data.keys())) - 1
cells = all_cells[np.logical_and(sig > min_sig, pv_ndnf3 > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv_ndnf3[cell], decimals = 2)))


Cell 120: depth 306.89um, signal = 0.27, p value = 0.5
Cell 44: depth 253.44um, signal = 1.11, p value = 0.5
Cell 27: depth 258.68um, signal = 0.11, p value = 0.67
Cell 120: depth 306.89um, signal = 0.27, p value = 0.5
Cell 120: depth 306.89um, signal = 0.27, p value = 0.5
Cell 120: depth 306.89um, signal = 0.27, p value = 0.5
Cell 27: depth 258.68um, signal = 0.11, p value = 0.67
Cell 44: depth 253.44um, signal = 1.11, p value = 0.5
Cell 27: depth 258.68um, signal = 0.11, p value = 0.67
Cell 27: depth 258.68um, signal = 0.11, p value = 0.67


## Vip+ cells

In [78]:
sig = int_vip_bin - avg_bg_vip_bin
pv = pv_vip3
max_sig = 10
all_cells = np.array(list(Cell.cell_data.keys())) - 1
cells = all_cells[np.logical_and(sig < max_sig, pv < 0.05)]
# cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv[cell], decimals = 2)))


Cell 1: depth 93.79um, signal = 2.83, p value = 0.0
Cell 15: depth 164.33um, signal = 5.74, p value = 0.0
Cell 20: depth 286.32um, signal = 0.64, p value = 0.0
Cell 25: depth 95.87um, signal = 3.86, p value = 0.0
Cell 33: depth 100.37um, signal = 2.53, p value = 0.0
Cell 39: depth 206.94um, signal = 2.27, p value = 0.0
Cell 44: depth 253.44um, signal = 0.54, p value = 0.0
Cell 63: depth 229.51um, signal = 9.76, p value = 0.0
Cell 70: depth 382.53um, signal = 8.9, p value = 0.0
Cell 84: depth 351.77um, signal = 1.42, p value = 0.0
Cell 100: depth 104.38um, signal = 0.1, p value = 0.0
Cell 113: depth 244.89um, signal = 5.3, p value = 0.0
Cell 120: depth 306.89um, signal = 1.9, p value = 0.0
Cell 125: depth 366.83um, signal = 1.83, p value = 0.0
Cell 127: depth 332.13um, signal = 0.9, p value = 0.0
Cell 162: depth 176.45um, signal = 3.31, p value = 0.0
Cell 167: depth 160.16um, signal = 1.69, p value = 0.0
Cell 177: depth 143.9um, signal = -0.24, p value = 0.0
Cell 194: depth 371.96um, si

## Vip- cells

In [79]:
sig = int_vip_bin - avg_bg_vip_bin
pv = pv_vip3
max_sig = 10
all_cells = np.array(list(Cell.cell_data.keys())) - 1

cells = all_cells[np.logical_and(sig < max_sig, pv > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv[cell], decimals = 2)))


Cell 330: depth 65.82um, signal = 0.0, p value = 1.0
Cell 142: depth 91.81um, signal = 0.0, p value = 1.0
Cell 258: depth 356.49um, signal = -2.47, p value = 1.0
Cell 338: depth 347.63um, signal = 0.0, p value = 1.0
Cell 137: depth 211.21um, signal = 0.0, p value = 1.0
Cell 310: depth 293.24um, signal = 0.0, p value = 1.0
Cell 207: depth 326.26um, signal = 0.0, p value = 1.0
Cell 66: depth 296.52um, signal = 0.0, p value = 1.0
Cell 347: depth 12.87um, signal = 0.0, p value = 1.0
Cell 181: depth 248.89um, signal = 0.0, p value = 1.0


In [80]:
sig = int_vip_bin - avg_bg_vip_bin
pv = pv_vip3
max_sig = 0
all_cells = np.array(list(Cell.cell_data.keys())) - 1

cells = all_cells[np.logical_and(sig < max_sig, pv > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv[cell], decimals = 2)))


Cell 170: depth 143.46um, signal = -11.24, p value = 1.0
Cell 299: depth 257.23um, signal = -1.16, p value = 1.0
Cell 124: depth 210.6um, signal = -0.15, p value = 1.0
Cell 268: depth 219.07um, signal = -1.41, p value = 1.0
Cell 189: depth 151.63um, signal = -1.29, p value = 1.0
Cell 255: depth 174.82um, signal = -2.57, p value = 1.0
Cell 139: depth 302.68um, signal = -0.45, p value = 1.0
Cell 278: depth 200.48um, signal = -0.92, p value = 1.0
Cell 355: depth 373.07um, signal = -5.07, p value = 1.0
Cell 297: depth 180.76um, signal = -3.21, p value = 1.0


In [99]:
sig = int_vip_bin - avg_bg_vip_bin
pv = pv_vip3
min_sig = 0
all_cells = np.array(list(Cell.cell_data.keys())) - 1

cells = all_cells[np.logical_and(sig > min_sig, pv > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv[cell], decimals = 2)))


Cell 81: depth 242.36um, signal = 0.72, p value = 0.49
Cell 82: depth 321.98um, signal = 0.54, p value = 0.5
Cell 114: depth 320.9um, signal = 0.41, p value = 0.33
Cell 81: depth 242.36um, signal = 0.72, p value = 0.49
Cell 81: depth 242.36um, signal = 0.72, p value = 0.49
Cell 81: depth 242.36um, signal = 0.72, p value = 0.49
Cell 81: depth 242.36um, signal = 0.72, p value = 0.49
Cell 114: depth 320.9um, signal = 0.41, p value = 0.33
Cell 81: depth 242.36um, signal = 0.72, p value = 0.49
Cell 27: depth 258.68um, signal = 0.11, p value = 0.67


## Sst + cells

In [81]:
sig = int_sst_bin - avg_bg_sst_bin
pv = pv_sst3
max_sig = 10
all_cells = np.array(list(Cell.cell_data.keys())) - 1

cells = all_cells[np.logical_and(sig < max_sig, pv < 0.05)]
# cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv[cell], decimals = 2)))


Cell 105: depth 152.58um, signal = 1.08, p value = 0.0
Cell 109: depth 177.52um, signal = 0.46, p value = 0.0
Cell 121: depth 372.27um, signal = 0.93, p value = 0.0
Cell 132: depth 333.32um, signal = 1.4, p value = 0.0
Cell 166: depth 209.32um, signal = 0.34, p value = 0.0
Cell 201: depth 242.39um, signal = 5.43, p value = 0.0
Cell 316: depth 382.25um, signal = 5.72, p value = 0.0


## Sst- cells

In [83]:
sig = int_sst_bin - avg_bg_sst_bin
pv = pv_sst3
max_sig = 0
all_cells = np.array(list(Cell.cell_data.keys())) - 1

cells = all_cells[np.logical_and(sig < max_sig, pv > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv[cell], decimals = 2)))


Cell 14: depth 155.78um, signal = -0.44, p value = 1.0
Cell 14: depth 155.78um, signal = -0.44, p value = 1.0
Cell 14: depth 155.78um, signal = -0.44, p value = 1.0
Cell 29: depth 299.58um, signal = -0.15, p value = 1.0
Cell 14: depth 155.78um, signal = -0.44, p value = 1.0
Cell 14: depth 155.78um, signal = -0.44, p value = 1.0
Cell 29: depth 299.58um, signal = -0.15, p value = 1.0
Cell 29: depth 299.58um, signal = -0.15, p value = 1.0
Cell 29: depth 299.58um, signal = -0.15, p value = 1.0
Cell 14: depth 155.78um, signal = -0.44, p value = 1.0


In [84]:
sig = int_sst_bin - avg_bg_sst_bin
pv = pv_sst3
max_sig = 10
all_cells = np.array(list(Cell.cell_data.keys())) - 1

cells = all_cells[np.logical_and(sig < max_sig, pv > 0.05)]
cells = cells[np.random.randint(0, high = len(cells) - 1, size = 10)]
for cell in cells:
    print('Cell {0}: depth {1}um, signal = {2}, p value = {3}'.format(cell, 
                                                                      np.round(depths[cell], decimals = 2), 
                                                                      np.round(sig[cell], decimals = 2), 
                                                                      np.round(pv[cell], decimals = 2)))


Cell 112: depth 253.86um, signal = 0.0, p value = 1.0
Cell 118: depth 166.39um, signal = 0.0, p value = 1.0
Cell 62: depth 223.94um, signal = 0.0, p value = 1.0
Cell 104: depth 84.27um, signal = 0.0, p value = 1.0
Cell 98: depth 61.4um, signal = 0.0, p value = 1.0
Cell 207: depth 326.26um, signal = 0.0, p value = 1.0
Cell 353: depth 169.94um, signal = 0.0, p value = 1.0
Cell 248: depth 392.49um, signal = 0.0, p value = 1.0
Cell 317: depth 236.14um, signal = 0.0, p value = 1.0
Cell 122: depth 166.83um, signal = 0.0, p value = 1.0
